In [72]:
import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import os
import shutil
import requests
import datetime

import matplotlib.pyplot as plt
import matplotlib as mpl 
from matplotlib.pyplot import savefig
import time

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
mpl.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题

%matplotlib inline

## 数据准备

### 云文件目录

In [73]:
dir_dic   = 'E:\\SynologyDrive\\dir_dic\\'
dir_login = 'E:\\SynologyDrive\\dir_login\\'
dir_order = 'E:\\SynologyDrive\\dir_order\\'
dir_user  = 'E:\\SynologyDrive\\dir_user\\'
dir_order_life = 'E:\\SynologyDrive\\dir_order\\order_life\\'
dir_score = 'E:\\SynologyDrive\\dir_score\\'
dir_sup = 'E:\\SynologyDrive\\data_support_pro\\'
dir_report = 'E:\\SynologyDrive\\日报明细数据\\'
dir_down = 'D:\\User\\Downloads\\'
dir_des = 'D:\\User\Desktop\\'

In [74]:
date_bus = pd.read_excel(dir_dic  + '03_time_index.xlsx',parse_dates=['date_pay'])
area_name = pd.read_excel(dir_dic +  '00_用户区域中心对照表.xlsx')
area_target = pd.read_excel(dir_dic + '01_2020年各机构月度目标表.xlsx')
ptype = pd.read_excel(dir_dic + '04_产品映射表.xlsx')
mon_quart = pd.read_excel(dir_dic  + '08_月份_季度映射表.xlsx')
mon_sdate_map = pd.read_excel(dir_dic + '07_月份_当月开始时间映射表.xlsx')
covar = pd.read_excel(dir_dic + '11_保额字典.xlsx')
order_renew = pd.read_excel(dir_dic + '10_order_renew_new_core.xlsx')

In [75]:
d1= list(date_bus.date_pay)
d2 = list(date_bus.mon_business)
date_bus = dict(zip(d1,d2))


d1 = list(mon_sdate_map.mon)
d2 = list(mon_sdate_map.sdate)
mon_sdate_map= dict(zip(d1,d2))

d1 = list(mon_quart.mon)
d2 = list(mon_quart.q)
mon_quart = dict(zip(d1,d2))

def day_date_descision(model,start,end):
    start  =pd.to_datetime(start)
    end  =pd.to_datetime(end)
    if model == 1:
        day_date_start = start
        day_date_end = start
    else:
        day_date_start = start
        day_date_end = end
    return day_date_start,day_date_end
def getAge(x):
    try :
        bornYear = int(x[6:10])
        age = 2020-bornYear
        return age
    except(TypeError,ValueError):
        return -1
def getGender(x):
    try :
        gender = int(x[-2:-1])
        if gender %2 ==1:
            return '男'
        else:
            return '女'
    except(TypeError,ValueError):
        return '未知'

## 临时数据

In [150]:
col = ['pname','date_pay','bf','uid','is_insider','order']
df = pd.concat([order_20[col],order_2019_need[col]])

In [151]:
df['mon'] = df.date_pay.map(date_bus)

In [152]:
df['is_renew'] = np.nan
df.loc[df.order.isin(order_renew.order),'is_renew'] ='续购'
df.is_renew.fillna('新单',inplace=True)

In [141]:
df.groupby(['mon','is_insider']).uid.nunique().reset_index().pivot('mon','is_insider','uid')

is_insider,否,是
mon,,
2019-01,9074,14270
2019-02,7887,13684
2019-03,10688,15600
2019-04,10642,15283
2019-05,9773,14679
2019-06,10159,14855
2019-07,9744,13707
2019-08,10366,14269
2019-09,12490,16004


In [142]:
pt = pd.read_clipboard()

In [153]:
res = pd.merge(df,pt,how='left',on='pname')

In [161]:
res.groupby(['mon','plabel_L2','pname','is_renew']).bf.sum().reset_index().to_excel('12.xlsx')

### 用户数据

In [24]:
user_info_all = pd.read_excel('D:\\User\\Desktop\\user_info\\sensor\\user_info_20191226.xlsx')

In [44]:
user_all = pd.read_csv(dir_user + 'user_info_online_20200429.csv',encoding='utf-8',low_memory=False)

In [35]:
user_01 = pd.read_excel(dir_down + '用户管理 (21).xlsx')
user_02 = pd.read_excel(dir_down + '用户管理 (22).xlsx')
user = pd.concat([user_01,user_02])

In [37]:
user.区域中心.replace('大童|区域中心','',regex=True,inplace=True)

In [39]:
user[['用户ID','代理人编码','区域中心']].to_excel('12.xlsx')

In [105]:
user_info = pd.read_excel(dir_user + 'user_info_offline_20200426.xlsx',header=2)

### 历史数据

In [4]:
order_2018 = pd.read_csv(dir_order + 'order_2018.csv',low_memory=False,parse_dates=['date_pay'])
# order_2017 = pd.read_csv(dir_order + 'order_year\\csv\\order_2017.csv',low_memory=False,parse_dates=['date_pay'])

In [5]:
order_2019 = pd.read_csv(dir_order + 'order_detail_2019_1226_1226.csv',encoding='utf-8',low_memory=False,parse_dates=['date_pay'])
ct_order = pd.read_excel(dir_order + 'ct_order_0426.xlsx')  
ct_order_list = list(ct_order.order.unique())
order_stat_need = ['已生效','待生效','已发货' '已收货', '提取NDA','核保中']
order_2019_need = order_2019.loc[ (order_2019.order_stat.isin(order_stat_need))
                                & (order_2019.order.isin(ct_order_list)==False),:].copy()

In [22]:
order_2019_need['mon'] = order_2019_need.date_pay.map(date_bus)

### 今日数据

In [76]:
%%time
order_2020_all = pd.read_csv('D:\\User\\Desktop\\order_info\\order_detail_2020_0524.csv',encoding='utf-8',low_memory=False,parse_dates=['date_pay'])

Wall time: 15.6 s


In [77]:
order_stat_need = ['已生效','待生效','已发货']
order_sum = order_2020_all.loc[(order_2020_all.order_stat.isin(order_stat_need)),:].copy()
order_sum['mon'] = order_sum.date_pay.map(date_bus)
order_sum.groupby('date_pay').bf.sum().reset_index().tail()

,date_pay,bf
146,2020-05-20,1860189.58
147,2020-05-21,1409078.52
148,2020-05-22,1396209.14
149,2020-05-23,1129973.66
150,2020-05-24,1072586.31


### BI平台业务数据

In [88]:
df6 = pd.read_excel(dir_down + '互联网业务清单 (6).xlsx',header = 2)
df5 = pd.read_excel(dir_down + '互联网业务清单 (5).xlsx',header = 2)
df4 = pd.read_excel(dir_down + '互联网业务清单 (4).xlsx',header = 2)
df3 = pd.read_excel(dir_down + '互联网业务清单 (3).xlsx',header = 2)
df2 = pd.read_excel(dir_down + '互联网业务清单 (2).xlsx',header = 2)
df7 = pd.read_excel(dir_down + '互联网业务清单 (7).xlsx',header = 2)
df9 = pd.read_excel(dir_down + '互联网业务清单 (9).xlsx',header = 2)
df10 = pd.read_excel(dir_down + '互联网业务清单 (10).xlsx',header = 2)
df11 = pd.read_excel(dir_down + '互联网业务清单 (11).xlsx',header = 2)
df12 = pd.read_excel(dir_down + '网销业务清单.xlsx',header = 2)

In [89]:
df = pd.concat([df2,df3,df4,df5,df6,df7,df9,df10,df11,df12]).drop_duplicates()

In [90]:
df.rename(columns = {'支付日期':'time_pay','保费':'bf','产品名称':'pname','区域中心':'area_type','核心编码':'ucode',
                     '缴费年期':'year_pay','保障期间':'year_protect','订单号':'order'},inplace=True)

In [91]:
df.order.replace('TW|T','',inplace=True,regex=True)

In [92]:
df['date_pay'] = pd.to_datetime(pd.to_datetime(df.time_pay).dt.strftime('%Y-%m-%d'))
df['mon'] = df.date_pay.map(date_bus)

ParserError: Unknown string format: 支付日期

In [30]:
order_all = pd.merge(df,ptype,how='left',on='pname')

In [31]:
order_all.loc['order_type'] = np.nan
order_all.loc[order_all.year_protect.isin(['至-105岁','至-99岁','至-88岁']),'order_type'] = '终身重疾'
order_all.order_type.fillna('消费重疾',inplace=True)

In [32]:
order_all.loc[(order_all.plabel_term==1) & (order_all.plabel_need =='重疾险_其他'),:].groupby(['order_type']).bf.sum().reset_index()

,order_type,bf
0,消费重疾,2378963.35
1,终身重疾,15803805.30


In [43]:
order_all.loc[(order_all.plabel_term==1) & (order_all.plabel_need =='重疾险_其他'),:].groupby(['area_type','mon','order_type']).agg({'ucode':'nunique','order':'count','bf':'sum'}).reset_index().to_excel('12.xlsx')

In [69]:
df.loc[df.ucode ==640288623]

,序号,订单号,保单号,投保人,被保人,pname,bf,保额,缴费年期,保障期间,time_pay,ucode,姓名,area_type,保单状态,date_pay
39110,39111,202004131008517116,10103006600038406349,胡方苑,胡前平,平安个人意外保障计划,375.0,0.0,1年,NaN,2020-04-13 21:17:06,640288623.0,王静,大童安徽区域中心,已承保,2020-04-13


## 新日报

### 基本处理

In [78]:
fileList = []
for x in os.listdir(dir_down):
    str_code = x.find('快保经营日报')

    if str_code ==0:
        fileDate = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(os.stat(dir_down +x).st_mtime))
        total_name = '%s_%s'%(fileDate,x)
#         print(total_name)
        fileList.append(total_name)
file = sorted(fileList,reverse=True)[0]
file_name = file.split('_')[1]
# print(file)

nowTime=datetime.datetime.now().strftime('%Y-%m-%d')
shutil.copyfile(dir_down +file_name,dir_report+nowTime+file_name)
shutil.copyfile(dir_down +file_name,dir_report+'快保经营日报数据.xlsx')
print(file_name)

快保经营日报-业务 (16).xlsx


### 续购口径

In [79]:
model =2
start = '2020-05-22'
end = '2020-05-24'

tsdate = day_date_descision(model,start,end)[0]
tedate = day_date_descision(model,start,end)[1]

curmon = '2020-05'
msdate = mon_sdate_map[curmon]
medate = tedate

ysdate = pd.to_datetime('2019-12-26')
yedate = tedate

print('当日数据时间%s'%tedate)
print('当月开始时间%s,当月结束时间%s'%(msdate,medate))
print('当月开始时间%s,当月结束时间%s'%(ysdate,yedate))

当日数据时间2020-05-24 00:00:00
当月开始时间2020-04-26 00:00:00,当月结束时间2020-05-24 00:00:00
当月开始时间2019-12-26 00:00:00,当月结束时间2020-05-24 00:00:00


In [81]:
order_20 = pd.merge(order_sum,ptype,how='left',on='pname')
order_19 = pd.merge(order_2019_need,ptype,how='left',on='pname')

In [82]:
need_plabel = ['健康险_百万医疗','健康险_专项医疗','健康险_住院保','健康险_门诊住院','健康险_海外医疗',
               '意外险_“10+1”','意外险_高额意外','意外险_学平险','意外险_特殊人群']
order_20['days_protect'] = (pd.to_datetime(order_20.time_end) - pd.to_datetime(order_20.time_start)).map(lambda x:x.days)
order_19['days_protect'] = (pd.to_datetime(order_19.time_end) - pd.to_datetime(order_19.time_start)).map(lambda x:x.days)

order_19_renew = order_19.loc[(order_19.days_protect.isin([364,365])) & (order_19.plabel.isin(need_plabel)),:].copy()
order_20_renew = order_20.loc[(order_20.days_protect.isin([364,365])) & (order_20.plabel.isin(need_plabel)),:].copy()

In [83]:
# order_renew_all['mon'] = order_renew_all.date_pay_x.map(date_bus)
order_renew_all = pd.merge(order_20_renew,order_19_renew,how='left',on=['bb_id','plabel'],suffixes=['_ty','_ly'])
order_renew_all['delta_renew'] = (pd.to_datetime(order_renew_all.time_end_ly) - pd.to_datetime(order_renew_all.time_start_ty)).map(lambda x:x.days)

In [84]:
col = ['bb_id','pname_ty','pname_ly','time_end_ly','time_start_ty','delta_renew','order_renew_src_ty','order_ty','bf_ty','date_pay_ty','order_ly']
order_renew_need = order_renew_all.loc[(order_renew_all.pname_ly.notna())
                                       &(order_renew_all.delta_renew <30)
                                       & (order_renew_all.delta_renew >=-30),col].copy()
order_renew_need.tail()

,bb_id,pname_ty,pname_ly,time_end_ly,time_start_ty,delta_renew,order_renew_src_ty,order_ty,bf_ty,date_pay_ty,order_ly
455567,411024198606096233,亚太金典综合保障计划,平安个人意外保障计划,2020-05-29,2020-05-29,0.0,NaN,202005221087675688,200.0,2020-05-22,201905236721058284
455570,532301198801040016,亚太守护意外保,亚太守护意外保,2020-05-24,2020-05-25,-1.0,同类型产品库续购,202005224666126724,258.0,2020-05-22,201905224858752006
455575,321181198707113187,众惠全能百万医疗2019款,众惠全能百万医疗保险,2020-05-24,2020-05-25,-1.0,续投-APP,202005221975094373,366.0,2020-05-22,201905211106788431
455577,321181198608295974,众惠全能百万医疗2019款,众惠全能百万医疗保险,2020-05-24,2020-05-25,-1.0,续投-APP,202005229849241200,366.0,2020-05-22,201905212649183537
455580,133023198001190241,众安尊享e生2020版,众惠全能百万医疗2019款,2020-05-15,2020-05-23,-8.0,NaN,202005224666113143,473.0,2020-05-22,201905152393170580


In [85]:
renew_col =['保险公司回传', '续投-短信C', '续投-分享', '续投-APP', '续投-C端短信', '续投-邮件','续投-短信A', '续投-微信公众号']
order_renew_01 = order_renew_need[['order_ty']]
order_renew_02 = order_renew_all.loc[order_renew_all.order_renew_src_ty.isin(renew_col),['order_ty']]
order_renew_res = pd.concat([order_renew_01,order_renew_02]).drop_duplicates()

In [86]:
order_20['is_renew'] = np.nan
order_20.loc[order_20.order.isin(order_renew_res.order_ty.unique()),'is_renew'] = '续购'
order_20.is_renew.fillna('新单',inplace=True)

In [69]:
order_20.groupby(['mon','is_renew']).bf.sum().reset_index().tail()

,mon,is_renew,bf
5,2020-03,续购,13817539.46
6,2020-04,新单,29056514.82
7,2020-04,续购,14354903.06
8,2020-05,新单,25546153.31
9,2020-05,续购,10607653.00


In [94]:
col = ['order','bf','uid','date_pay','ucode','area_type']
order_20.loc[order_20.pname=='复星联合爱无忧意外伤害保险',col].to_excel('12.xlsx')

### 区域中心续购新单数据输出

In [87]:
order_20['area_type_need'] = order_20.area_type.copy()
order_20.loc[(order_20.area_type =='大童直属区域中心') 
              | (order_20.area_type.isna()),'area_type_need'] = '其他'
order_20.area_type_need.replace('大童|区域中心','',regex=True,inplace=True)
order_20.loc[order_20.area_type_need =='直属经纪','area_type_need'] = '经纪北京'
order_20.loc[order_20.area_type_need =='内蒙古','area_type_need'] = '内蒙'

In [88]:
order_d = order_20.loc[(order_20.date_pay >= tsdate) &(order_20.date_pay<=tedate) & (order_20.is_renew =='续购'),:]\
.groupby(['area_type_need']).bf.sum().reset_index()
order_m = order_20.loc[(order_20.date_pay >= msdate) &(order_20.date_pay<=medate) & (order_20.is_renew =='续购'),:]\
.groupby(['area_type_need']).bf.sum().reset_index()
order_y = order_20.loc[(order_20.date_pay >= ysdate) &(order_20.date_pay<=yedate) & (order_20.is_renew =='续购'),:]\
.groupby(['area_type_need']).bf.sum().reset_index()

In [89]:
res_mid = pd.merge(order_y,order_m,how='left',on='area_type_need',suffixes=['_y','_m'])
res = pd.merge(res_mid,order_d,how='left',on='area_type_need')[['area_type_need','bf','bf_m','bf_y']]
res.to_excel(dir_report + '各区域中心续购情况.xlsx')

### E营业部报表

In [95]:
order_20.loc[(order_20.team_bus_name.str.contains('E')) 
             & (order_20.team_bus_name.notna()),:].groupby(['date_pay','area_type','team_bus_name','uname','pname']).bf.sum().reset_index().sort_values('date_pay',ascending=False)

,date_pay,area_type,team_bus_name,uname,pname,bf
54,2020-05-23,大童河北区域中心,河北E营业部,冯小建,华贵大麦2020定期寿险,1498.8
53,2020-05-21,大童辽宁区域中心,辽宁E营业部,王卓,国寿财学平险2019版,70.0
52,2020-05-21,大童河北区域中心,河北E营业部,林莉,平安中老年意外保障计划,209.0
51,2020-05-20,大童湖北区域中心,湖北E营业部,鲍铭涵,泰康住院保2019-老年版,419.0
50,2020-05-20,大童湖北区域中心,湖北E营业部,鲍铭涵,泰康住院保2019-成人版,478.0
49,2020-05-20,大童湖北区域中心,湖北E营业部,鲍铭涵,众安尊享e生2019版,1948.0
48,2020-05-20,大童浙江区域中心,浙江E营业部,赵欣怡,平安中老年意外保障计划,309.0
47,2020-05-20,大童浙江区域中心,浙江E营业部,赵欣怡,华贵大麦2020定期寿险,1338.5
46,2020-05-20,大童河北区域中心,河北E营业部,陈丹霞,众安尊享e生2019版,932.0
45,2020-05-20,大童安徽区域中心,安徽E营业部,杨茂杰,百年康惠保(2020版)重大疾病保险,547.0


### 险种变化趋势图

In [30]:
order_20['date_pay_str'] = order_20.date_pay.dt.strftime('%m-%d')

In [31]:
order_20.loc[(order_20.plabel_L1.isin(['意外险','重疾险','健康险','定期寿险'])) 
             & (order_20.date_pay>= pd.to_datetime('2020-03-01'))].groupby(['date_pay_str','plabel_need']).bf.sum().reset_index().pivot(index='date_pay_str',columns ='plabel_need',values='bf').tail(30).to_excel('12.xlsx')

### 邮件发送数据

In [96]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

export_en_col = ['order','order_origin','pname','bf','date_pay','ucode','uname','area_type']
export_cn_col = ['订单号','保单号','产品名称','保费','支付日期','核心编码','姓名','区域中心']


_user = "liusf@dtinsure.com"
_pwd  = "Lausf621"
dir_mail = 'E:\\SynologyDrive\\data_support_res\\'
## 经纪
order_bj = order_sum.loc[(order_sum.date_pay >= pd.to_datetime('2020-04-26'))
             & (order_sum.area_type =='大童直属经纪区域中心'),export_en_col].copy()
order_bj.columns = export_cn_col
order_bj.to_excel(dir_mail+'2020年经纪北京互联网报表.xlsx',index=0)                 
file_bj = dir_mail + '2020年经纪北京互联网报表.xlsx'

to_bj   = "ningsai@dtinsure.com"
msg = MIMEMultipart()
msg["Subject"] = "互联网保单明细"
msg["From"]    = _user
msg["To"]      = to_bj

part = MIMEText("互联网保单明细，请查收！")
msg.attach(part)

#---这是附件部分---
#xlsx类型附件
part = MIMEApplication(open(file_bj,'rb').read())
part.add_header('Content-Disposition', 'attachment', filename="2020年经纪北京互联网保单明细.xlsx")
msg.attach(part)

s = smtplib.SMTP("smtp.dtinsure.com", 25) #连接smtp邮件服务器,端口默认是25
s.login(_user, _pwd) #登陆服务器
s.sendmail(_user, to_bj, msg.as_string()) #发送邮件
s.close()
print('邮件发送成功')
del part

邮件发送成功


## 家庭线上线下产品搭配情况

### 基础数据处理

In [168]:
res = order_zj_on[['tb_id','bb_id']].copy()
node = []
for x in res.values:
#     print(tuple(x))
    node.append(tuple(x))

In [94]:
order_20_need = pd.merge(order_20,df[['order','保额','year_pay','year_protect']],how='left',on='order')

In [95]:
order_zj_on = order_20_need.loc[(order_20_need.plabel_L1 =='重疾险') & (order_20.plabel_term ==1),:].copy()
order_off_sum.rename(columns = {'date_input':'date_pay','投保单号':'order'},inplace=True)
order_zj_off = order_off_sum.copy()

In [96]:
order_zj_on['order_type'] = '线上重疾'
order_zj_off['order_type'] = np.nan
order_zj_off.loc[order_zj_off.pname.str.contains('重大疾病'),'order_type'] = '线下重疾'
order_zj_off.order_type.fillna('线下其他',inplace=True)

In [97]:
order_zj_on['zj_type'] = np.nan
order_zj_on.loc[order_zj_on.year_protect =='至-105岁','zj_type'] = '保终身'
order_zj_on.zj_type.fillna('保定期',inplace=True)
order_zj_off['zj_type'] = np.nan
order_zj_off.loc[order_zj_off.year_protect.isin(['至200岁','200年','至100岁','至105岁', '至106岁','至99岁', '至120岁', '100年']),'zj_type'] = '保终身'
order_zj_off.zj_type.fillna('保定期',inplace=True)

In [44]:
col = ['order','date_pay','pname','bf','ucode_need','tb_id','bb_id','relation','year_pay','year_protect','保额','order_type']
order_all = pd.concat([order_zj_on[col],order_zj_off[col]])

In [45]:
cust_zj_on = list(order_zj_on.tb_id.unique())
cust_zj_on.extend(list(order_zj_on.bb_id.unique()))
cust_zj_on = list(set(cust_zj_on))

In [46]:
cust_off = list(order_zj_off.tb_id.unique())
cust_off.extend(list(order_zj_off.bb_id.unique()))
cust_off = list(set(cust_off))

In [47]:
cust_zj_off = list( order_zj_off.loc[order_zj_off.order_type =='线下重疾'].tb_id.unique())
cust_zj_off.extend(list(order_zj_off.loc[order_zj_off.order_type =='线下重疾'].bb_id.unique()))
cust_zj_off = list(set(cust_zj_off))

### 统计-同一被保人配置情况

In [325]:
res = order_all.groupby(['bb_id','pname']).agg({'order':'count','bf':'sum'}).reset_index().pivot('bb_id','pname','order').reset_index()

In [315]:
res.loc[res.线上重疾.notna(),:].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12564 entries, 32 to 58150
Data columns (total 4 columns):
bb_id    12564 non-null object
线上重疾     12564 non-null float64
线下其他     902 non-null float64
线下重疾     2295 non-null float64
dtypes: float64(3), object(1)
memory usage: 490.8+ KB


In [281]:
order_zj_off.loc[order_zj_off.order_type =='线下重疾'].tb_id.nunique()

30049

In [282]:
order_zj_off.loc[order_zj_off.order_type =='线下重疾'].bb_id.nunique()

35714

In [76]:
order_zj_off['is_on_cust'] = np.nan
order_zj_off.loc[(order_zj_off.tb_id.isin(cust_zj_on)) | (order_zj_off.bb_id.isin(cust_zj_on)),'is_on_cust'] = '线上客户'
order_zj_off.is_on_cust.fillna('其他客户',inplace=True)

In [288]:
len(cust_zj_on)

19926

In [297]:
len(cust_off)

65726

In [295]:
len(cust_zj_off)

48903

In [299]:
cust_zj_on.extend(cust_zj_off)

In [302]:
len(set(cust_zj_on))

64182

### 线上重疾组合

#### 被保人视角

In [49]:
order_zj_on['order_com']  = order_zj_on.zj_type + '_' + order_zj_on.pname
order_zj_off['order_com']  = order_zj_off.zj_type + '_' + order_zj_off.pname
bb_on = order_zj_on.groupby(['bb_id']).order_com.unique().reset_index()
bb_off = order_zj_off.groupby(['bb_id']).order_com.unique().reset_index()

In [50]:
res = pd.merge(bb_on,bb_off,how='left',on='bb_id')

In [51]:
res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12645 entries, 0 to 12644
Data columns (total 3 columns):
bb_id          12645 non-null object
order_com_x    12645 non-null object
order_com_y    2697 non-null object
dtypes: object(3)
memory usage: 395.2+ KB


In [52]:
res['order_com_x'] = res.order_com_x.map(lambda x:str(x))

In [57]:
bb_list = res.loc[res.order_com_y.notna(),:].bb_id.unique()

In [378]:
order_zj_on.loc[order_zj_on.bb_id.isin(bb_list),:].groupby('zj_type').order.count().reset_index()

,zj_type,order
0,保定期,2052
1,保终身,829


In [379]:
order_zj_off.loc[order_zj_off.bb_id.isin(bb_list),:].groupby('zj_type').order.count().reset_index()

,zj_type,order
0,保定期,557
1,保终身,2989


In [59]:
order_on_com = order_zj_on.loc[order_zj_on.bb_id.isin(bb_list),:].groupby(['bb_id','order_com']).order.count().reset_index()
order_off_com = order_zj_off.loc[order_zj_off.bb_id.isin(bb_list),:].groupby(['bb_id','order_com']).order.count().reset_index()

In [62]:
pd.merge(order_on_com,order_off_com,how='left',on='bb_id').groupby(['order_com_x','order_com_y']).size().reset_index().to_excel('12.xlsx')

In [ ]:
## 只做线上线下的业务没有 交叉的良好
## 线上与线下是互为补充的关系。 

In [381]:
order_zj_on.groupby('zj_type').order.count().reset_index()

,zj_type,order
0,保定期,8139
1,保终身,5258


In [53]:
order_zj_off.groupby('zj_type').order.count().reset_index()

,zj_type,order
0,保定期,9185
1,保终身,51413


#### 投被家庭视角

In [66]:
order_fam = order_zj_off.loc[(order_zj_off.tb_id.isin(cust_zj_on)) | (order_zj_off.bb_id.isin(cust_zj_on)),:]
order_fam.groupby('zj_type').order.count().reset_index()

,zj_type,order
0,保定期,898
1,保终身,5079


In [67]:
fam_id = list(order_fam.tb_id.unique())
fam_id.extend(list(order_fam.bb_id.unique()))
fam_id = list(set(fam_id))

In [68]:
order_zj_on.loc[(order_zj_on.tb_id.isin(fam_id)) | (order_zj_on.bb_id.isin(fam_id)),:].groupby('zj_type').order.count().reset_index()

,zj_type,order
0,保定期,2970
1,保终身,1319


In [77]:
order_zj_off['is_on_cust'] = np.nan
order_zj_off.loc[(order_zj_off.tb_id.isin(cust_zj_on)) | (order_zj_off.bb_id.isin(cust_zj_on)),'is_on_cust'] = '线上客户'
order_zj_off.is_on_cust.fillna('其他客户',inplace=True)

In [79]:
ucode_on_off = order_zj_off.loc[order_zj_off.is_on_cust =='线上客户',:].ucode_need

In [87]:
order_zj_off.loc[(order_zj_off.ucode_need.isin(ucode_on_off))
                 & (order_zj_off.order_type =='线下重疾'),:].groupby('ucode_need').bf.sum().reset_index().describe()

,bf
count,2163.000000
mean,33249.909519
std,40364.029019
min,190.000000
25%,8714.500000
50%,19775.400000
75%,41873.250000
max,340390.980000


In [98]:
a1 = list(order_zj_off.loc[(order_zj_off.ucode_need.isin(ucode_on_off))
                 & (order_zj_off.order_type =='线下重疾'),:].tb_id.unique())
a2 = list(order_zj_off.loc[(order_zj_off.ucode_need.isin(ucode_on_off))
                 & (order_zj_off.order_type =='线下重疾'),:].bb_id.unique())
a1.extend(a2)
len(set(a1))

15314

In [99]:
a1 = list(order_zj_off.loc[(order_zj_off.ucode_need.isin(ucode_on_off))
                 & (order_zj_off.order_type =='线下重疾') & (order_zj_off.is_on_cust =='线上客户'),:].tb_id.unique())
a2 = list(order_zj_off.loc[(order_zj_off.ucode_need.isin(ucode_on_off))
                 & (order_zj_off.order_type =='线下重疾') & (order_zj_off.is_on_cust =='线上客户'),:].bb_id.unique())
a1.extend(a2)
len(set(a1))

5587

In [97]:
order_zj_off.loc[(order_zj_off.ucode_need.isin(ucode_on_off))
                 & (order_zj_off.order_type =='线下重疾'),:].tb_id.unique() + order_zj_off.loc[(order_zj_off.ucode_need.isin(ucode_on_off))& (order_zj_off.order_type =='线下重疾'),:].bb_id.unique()

ValueError: operands could not be broadcast together with shapes (9295,) (11210,) 

In [91]:
order_zj_on.loc[order_zj_on.ucode_need.isin(ucode_on_off),:].groupby('ucode_need').bf.sum().reset_index().describe()

,bf
count,2204.000000
mean,6544.423798
std,9973.005807
min,86.000000
25%,1123.000000
50%,3128.500000
75%,7909.400000
max,156573.000000


##  ABC重疾用户分类

In [103]:
order_zj_offline = order_zj_off.loc[order_zj_off.order_type =='线下重疾',:].copy()

In [104]:
res = pd.merge(order_zj_on.groupby('ucode_need').bf.sum().reset_index(),order_zj_offline.groupby('ucode_need').bf.sum().reset_index(),how='left',on='ucode_need')

In [107]:
order_zj_on['zj_type_01'] = order_zj_on.zj_type
order_zj_on.loc[order_zj_on.plabel_L2 =='少儿','zj_type_01'] = '少儿重疾'

In [106]:
ucode_on_off = res.loc[res.bf_y.notna(),:].ucode_need.unique()

In [124]:
agg_col_01 = {'uid':'nunique','bf':'sum'}
agg_col_02 = {'ucode_need':'nunique','bf':'sum'}

In [134]:
order_zj_on.loc[order_zj_on.ucode_need.isin(ucode_on_off)==False].groupby('zj_type_01').agg(agg_col_01).reset_index()

,zj_type_01,uid,bf
0,保定期,394,1740903.20
1,保终身,893,6703051.85
2,少儿重疾,1492,3048020.00


In [135]:
order_zj_on.loc[order_zj_on.ucode_need.isin(ucode_on_off)==False].agg(agg_col_01).reset_index()

,index,0
0,uid,2352.00
1,bf,11491975.05


In [136]:
order_zj_on.uid.nunique()

5725

In [132]:
order_zj_offline.loc[order_zj_offline.ucode_need.isin(ucode_on_off)==False].groupby('zj_type').agg(agg_col_02)

,ucode_need,bf
zj_type,,
保定期,202,8.422541e+05
保终身,7741,1.180733e+08


In [133]:
order_zj_offline.loc[order_zj_offline.ucode_need.isin(ucode_on_off)==False].agg(agg_col_02).reset_index()

,index,0
0,ucode_need,7.797000e+03
1,bf,1.189156e+08


In [131]:
len(ucode_on_off)

3395

## 客户首单产品

In [26]:
col =['tb_id','bb_id','pname','date_pay']
order_all = pd.concat([order_20[col],order_2019_need[col]])

In [27]:
order_all.date_pay.fillna(pd.to_datetime('2019-10-01'),inplace=True)

In [28]:
order_all['rns'] = order_all['date_pay'].groupby(order_all['tb_id']).rank(ascending=True,method='first')
tb_id_first = order_all.loc[order_all.rns ==1,['tb_id','date_pay','pname']].copy()

order_all['rns'] = order_all['date_pay'].groupby(order_all['bb_id']).rank(ascending=True,method='first')
bb_id_first = order_all.loc[order_all.rns ==1,['bb_id','date_pay','pname']].copy()

tb_id_first.columns = ['bb_id','date_pay','pname']
cust_all = pd.concat([tb_id_first,bb_id_first])

cust_all['rns'] =  cust_all['date_pay'].groupby(cust_all['bb_id']).rank(ascending=True,method='first')
cust_first = cust_all.loc[cust_all.rns ==1,:].copy()

In [29]:
plist = ['复星联合爱无忧意外伤害保险','安联女性特定重疾保险计划（18-40周岁）','安联女性特定重疾保险计划（41-50周岁）']
plist1 = ['复星联合爱无忧意外伤害保险']
plist2 = ['安联女性特定重疾保险计划（18-40周岁）','安联女性特定重疾保险计划（41-50周岁）']

In [30]:
cust_first.loc[cust_first.pname.isin(plist2),:].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57560 entries, 208117 to 350245
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   bb_id     57560 non-null  object        
 1   date_pay  57560 non-null  datetime64[ns]
 2   pname     57560 non-null  object        
 3   rns       57560 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.2+ MB


In [31]:
cust_first.loc[cust_first.pname.isin(plist1),:].head()

,bb_id,date_pay,pname,rns
141670,460022199210313916,2020-02-10,复星联合爱无忧意外伤害保险,1.0
141678,362121195808130021,2020-02-10,复星联合爱无忧意外伤害保险,1.0
141679,445281198803012710,2020-02-10,复星联合爱无忧意外伤害保险,1.0
141685,430981199704198317,2020-02-10,复星联合爱无忧意外伤害保险,1.0
141687,460021196302215213,2020-02-10,复星联合爱无忧意外伤害保险,1.0


In [32]:
cust_first.loc[cust_first.pname.isin(plist2),:].head()

,bb_id,date_pay,pname,rns
208117,421081199402173448,2020-02-25,安联女性特定重疾保险计划（18-40周岁）,1.0
211011,150204199204121286,2020-02-26,安联女性特定重疾保险计划（18-40周岁）,1.0
211012,422427197708091520,2020-02-26,安联女性特定重疾保险计划（41-50周岁）,1.0
211014,362523198501234422,2020-02-26,安联女性特定重疾保险计划（18-40周岁）,1.0
211015,321121199904034822,2020-02-26,安联女性特定重疾保险计划（18-40周岁）,1.0


In [33]:
cust_love = cust_first.loc[cust_first.pname.isin(plist1),:].bb_id.unique()
cust_female = cust_first.loc[cust_first.pname.isin(plist2),:].bb_id.unique()

### 净增客户转化

In [67]:
order_off_sum.columns

Index(['序号', '保单状态', 'date_tb', '回单日期', '保单录入日期', '承保日期', '预收日期', 'date_pay',
       '销售团队代码', '销售团队名称', '地区分类', '团队分类', '是否本部', '营业部代码', '营业部名称', '是否营业中心',
       '营业部所属批牌机构编码', '营业部所属批牌', '营业部类型', '代理人代码', '代理人姓名', '入司日期', '代理人职级',
       '业务合伙人级别', '经营合伙人级别', '经营团队总经理标志', '职业化标志', '推荐人代码', '推荐人姓名', '辅导人代码',
       '辅导人姓名', '经营合伙人代码', '经营合伙人姓名', '离职日期', 'ucode', '现代理人姓名', 'area_type',
       '原辅代理人姓名', '原辅代理人编码', '原辅代理人所在销售团队名称', '原辅代理人所在销售团队代码', '原辅代理人所在机构',
       '现辅代理人姓名', '现辅代理人编码', '现辅代理人所在销售团队名称', '现辅代理人所在销售团队代码', '是否自保件',
       '保险公司代码', '保险公司名称', '投保单号', '保单号', 'tb_name', 'bb_name', '险种代码',
       '外部险种代码', 'pname', '是否主险', '产品分类', '交费年期|年龄标志', '缴费期', '佣金率', '规模保费',
       'bf', '保额', '保全标志', '是否纳入现代理人方案统计', '归属类型', '归属细类', '保单关系人', '代码', '姓名',
       '月份', '销服代码', '销服名称', '机构', '职级', '入司日期.1', '入司月', '营业部代码.1', '营业部名称.1',
       '是否营业中心.1', '师傅代码', '师傅姓名', 'ucode_need', 'age', 'sex', 'area',
       'degree', 'grade', 'year_pay', 'year_protect', 'tb_gender', 'tb_age',
     

#### 爱无忧之后的保单

In [68]:
agg_col_01 = {'uid':'nunique','order':'count','bf':'sum'}
agg_col_02 = {'ucode_need':'nunique','order':'count','bf':'sum'}

on_col = ['order','pname','date_pay','bf','ucode_need','uid','tb_id','bb_id']
plist = ['复星联合爱无忧意外伤害保险','安联女性特定重疾保险计划（18-40周岁）','安联女性特定重疾保险计划（41-50周岁）']
off_col = ['order','pname','date_pay','bf','ucode_need','tb_id','bb_id']

order_off_sum.rename(columns = {'date_input':'date_pay','投保单号':'order'},inplace=True)

In [69]:
order_on =  order_20.loc[(order_20.pname.isin(plist)==False) & (order_20.date_pay >= pd.to_datetime('2020-02-10')),on_col].copy()
order_off = order_off_sum.loc[order_off_sum.date_pay >= pd.to_datetime('2020-02-10'),off_col].copy()

In [70]:
order_on['is_love_cust'] = np.nan
order_on.loc[(order_on.tb_id.isin(cust_love)) | (order_on.bb_id.isin(cust_love)),'is_love_cust'] = '爱无忧客户'
order_on.is_love_cust.fillna('未知客户',inplace=True)

In [71]:
order_on.loc[order_on.is_love_cust =='爱无忧客户',:].agg(agg_col_01)

uid         1418.00
order       2942.00
bf       1926260.74
dtype: float64

In [76]:
a1 = list(order_on.loc[order_on.is_love_cust =='爱无忧客户',:].tb_id.unique())
a2 = list(order_on.loc[order_on.is_love_cust =='爱无忧客户',:].bb_id.unique())
a1.extend(a2)
len(set(a1))

3227

In [72]:
order_off['is_love_cust'] = np.nan
order_off.loc[(order_off.tb_id.isin(cust_love)) | (order_off.bb_id.isin(cust_love)),'is_love_cust'] = '爱无忧客户'
order_off.is_love_cust.fillna('未知客户',inplace=True)

In [73]:
order_off.loc[order_off.is_love_cust =='爱无忧客户',:].agg(agg_col_02)

ucode_need         945.00
order             1758.00
bf            12445370.92
dtype: float64

In [77]:
a1 = list(order_off.loc[order_off.is_love_cust =='爱无忧客户',:].tb_id.unique())
a2 = list(order_off.loc[order_off.is_love_cust =='爱无忧客户',:].bb_id.unique())
a1.extend(a2)
len(set(a1))

1850

####  赠险之后的保单

In [78]:
order_on =  order_20.loc[(order_20.pname.isin(plist)==False) & (order_20.date_pay >= pd.to_datetime('2020-02-26')),on_col].copy()
order_off = order_off_sum.loc[order_off_sum.date_pay >= pd.to_datetime('2020-02-26'),off_col].copy()

In [80]:
order_on['is_female_cust'] = np.nan
order_on.loc[(order_on.tb_id.isin(cust_female)) | (order_on.bb_id.isin(cust_female)),'is_female_cust'] = '赠险客户'
order_on.is_female_cust.fillna('未知客户',inplace=True)
order_on.loc[order_on.is_female_cust =='赠险客户',:].agg(agg_col_01)

uid         1418.00
order       3231.00
bf       1747162.15
dtype: float64

In [82]:
a1 = list(order_on.loc[order_on.is_female_cust =='赠险客户',:].tb_id.unique())
a2 = list(order_on.loc[order_on.is_female_cust =='赠险客户',:].bb_id.unique())
a1.extend(a2)
len(set(a1))

3501

In [84]:
order_off['is_female_cust'] = np.nan
order_off.loc[(order_off.tb_id.isin(cust_female)) | (order_off.bb_id.isin(cust_female)),'is_female_cust'] = '赠险客户'
order_off.is_female_cust.fillna('未知客户',inplace=True)
order_off.loc[order_off.is_female_cust =='赠险客户',:].agg(agg_col_02)

ucode_need        979.00
order            1856.00
bf            9484858.28
dtype: float64

In [85]:
a1 = list(order_off.loc[order_off.is_female_cust =='赠险客户',:].tb_id.unique())
a2 = list(order_off.loc[order_off.is_female_cust =='赠险客户',:].bb_id.unique())
a1.extend(a2)
len(set(a1))

2046

#### 托管客户转化

In [91]:
cust_tg  = pd.read_csv(dir_order + 'order_tg.csv')

In [93]:
cust_tg.columns = ['uid','ucode','bb_id','time_tg']

In [96]:
cust_tg.loc[cust_tg.bb_id.isin(cust_love),:].agg({'uid':'nunique','bb_id':'nunique'})

uid      2316
bb_id    5454
dtype: int64

In [97]:
cust_tg.loc[cust_tg.bb_id.isin(cust_female),:].agg({'uid':'nunique','bb_id':'nunique'})

uid       3183
bb_id    12619
dtype: int64

### 所需客户

In [300]:
## 爱无忧客户 & 三八客户
on_col = ['order','pname','date_pay','bf','ucode_need','uid','tb_id','bb_id']
cust_love = order_20.loc[order_20.pname=='复星联合爱无忧意外伤害保险',on_col].copy()
cust_female = order_20.loc[order_20.pname.isin(['安联女性特定重疾保险计划（18-40周岁）','安联女性特定重疾保险计划（41-50周岁）']),on_col].copy()

In [ ]:
## 托管客户
cust_tg = pd.read

In [299]:
plist = ['复星联合爱无忧意外伤害保险','安联女性特定重疾保险计划（18-40周岁）','安联女性特定重疾保险计划（41-50周岁）']
order_on =  order_20.loc[(order_20.pname.isin(plist)==False) & (order_20.date_pay >= pd.to_datetime('2020-02-10')),on_col].copy()
off_col = ['投保单号','pname','date_input','bf','ucode_need','tb_id','bb_id']
order_off = order_off_sum.loc[order_off_sum.date_input >= pd.to_datetime('2020-02-10'),off_col].copy()

In [302]:
cust_love.uid.nunique()

9282

In [331]:
res = list(cust_love.tb_id.unique())
res.extend(list(cust_love.bb_id.unique()))
cust_love_list = pd.Series(res)
cust_love_list.nunique()

57787

In [333]:
order_on['is_love_cust'] = np.nan
order_on.loc[(order_on.tb_id.isin(cust_love_list)) | (order_on.bb_id.isin(cust_love_list)),'is_love_cust'] = '爱无忧客户'
order_on.is_love_cust.fillna('未知客户',inplace=True)

In [347]:
order_on.loc[order_on.is_love_cust =='爱无忧客户',:].groupby(['is_love_cust']).agg({'uid':'nunique','order':'count','bf':'sum'}).reset_index().sort_values('bf',ascending=False)

,is_love_cust,uid,order,bf
0,爱无忧客户,4527,14398,8385337.97


In [357]:
cust_female.uid.nunique()

7779

In [358]:
res = list(cust_female.tb_id.unique())
res.extend(list(cust_female.bb_id.unique()))
cust_female_list = pd.Series(res)
cust_female_list.nunique()

81208

In [343]:
order_on['is_female_cust'] = np.nan
order_on.loc[(order_on.tb_id.isin(cust_female_list)) | (order_on.bb_id.isin(cust_female_list)),'is_female_cust'] = '妇女节客户'
order_on.is_love_cust.fillna('未知客户',inplace=True)
order_on.groupby(['is_female_cust']).agg({'uid':'nunique','order':'count','bf':'sum'}).reset_index().sort_values('bf',ascending=False)

In [371]:
order_off['is_female_cust'] = np.nan
order_off.loc[(order_off.tb_id.isin(cust_female_list)) | (order_off.bb_id.isin(cust_female_list)),'is_female_cust'] = '妇女节客户'
order_off.is_female_cust.fillna('未知客户',inplace=True)
order_off.groupby(['is_female_cust']).agg({'ucode_need':'nunique','投保单号':'count','bf':'sum'}).reset_index().sort_values('bf',ascending=False)

,is_female_cust,ucode_need,投保单号,bf
1,未知客户,9526,32657,2.078104e+08
0,妇女节客户,2017,4266,2.082756e+07


In [369]:
plist = ['复星联合爱无忧意外伤害保险','安联女性特定重疾保险计划（18-40周岁）','安联女性特定重疾保险计划（41-50周岁）']
order_on =  order_20.loc[(order_20.pname.isin(plist)==False) & (order_20.date_pay >= pd.to_datetime('2020-02-25')),on_col].copy()
off_col = ['投保单号','pname','date_input','bf','ucode_need','tb_id','bb_id']
order_off = order_off_sum.loc[order_off_sum.date_input >= pd.to_datetime('2020-02-25'),off_col].copy()

In [372]:
order_off.loc[order_off.is_female_cust =='妇女节客户',:].groupby(['date_input']).agg({'ucode_need':'nunique','投保单号':'count','bf':'sum'}).reset_index().sort_values('date_input',ascending=True)

,date_input,ucode_need,投保单号,bf
0,2020-02-25,59,79,253774.80
1,2020-02-26,56,74,298639.16
2,2020-02-27,56,71,424797.72
3,2020-02-28,63,86,589269.94
4,2020-02-29,50,64,398800.32
5,2020-03-01,59,76,273923.02
6,2020-03-02,77,104,400541.25
7,2020-03-03,50,71,250117.92
8,2020-03-04,46,65,216644.35
9,2020-03-05,57,76,353532.84


### 托管转化

## 线上线下用户群

### 线下产品用户年龄

In [52]:
user_insider = pd.read_csv(dir_user + 'user_insider.csv',low_memory=False)
user_all = pd.read_csv(dir_user + 'user_info_all.csv',low_memory=False)
cust_all = pd.read_csv(dir_user + 'cust_user.csv',low_memory=False)

In [53]:
order_off_tb = pd.read_csv(dir_order_life + 'order_off_tb.csv')

In [54]:
order_off_tb.columns = ['投保单号','tb_id','bb_id','relation']

In [55]:
order_life_20.columns

Index(['序号', '保单状态', 'date_tb', '回单日期', '保单录入日期', '承保日期', '预收日期', 'date_pay',
       '销售团队代码', '销售团队名称', '地区分类', '团队分类', '是否本部', '营业部代码', '营业部名称', '是否营业中心',
       '营业部所属批牌机构编码', '营业部所属批牌', '营业部类型', '代理人代码', '代理人姓名', '入司日期', '代理人职级',
       '业务合伙人级别', '经营合伙人级别', '经营团队总经理标志', '职业化标志', '推荐人代码', '推荐人姓名', '辅导人代码',
       '辅导人姓名', '经营合伙人代码', '经营合伙人姓名', '离职日期', 'ucode', '现代理人姓名', 'area_type',
       '原辅代理人姓名', '原辅代理人编码', '原辅代理人所在销售团队名称', '原辅代理人所在销售团队代码', '原辅代理人所在机构',
       '现辅代理人姓名', '现辅代理人编码', '现辅代理人所在销售团队名称', '现辅代理人所在销售团队代码', '是否自保件',
       '保险公司代码', '保险公司名称', '投保单号', '保单号', 'tb_name', 'bb_name', '险种代码',
       '外部险种代码', 'pname', '是否主险', '产品分类', '交费年期|年龄标志', '缴费期', '佣金率', '规模保费',
       'bf', '保额', '保全标志', '是否纳入现代理人方案统计', '归属类型', '归属细类', '保单关系人', '代码', '姓名',
       '月份', '销服代码', '销服名称', '机构', '职级', '入司日期.1', '入司月', '营业部代码.1', '营业部名称.1',
       '是否营业中心.1', '师傅代码', '师傅姓名', 'ucode_need'],
      dtype='object')

In [56]:
order_life_20 = pd.read_excel(dir_order_life + 'order_life_20200514.xlsx',header=1)
order_life_20.rename(columns = {'行政录入日期':'date_pay' ,'投保人':'tb_name' ,'被保险人':'bb_name','退保日期':'date_tb'
                              ,'现代理人编码':'ucode','标准保费':'bf','代理人所在机构':'area_type','险种名称':'pname'},inplace=True)
order_life_20.ucode.fillna(0,inplace=True)
order_life_20['ucode_need'] = order_life_20.ucode.map(lambda x:str(int(x)).strip())

In [57]:

order_life_20 = order_life_20.loc[order_life_20.是否主险 =='Y',:].copy()

In [58]:
user_insider.columns =['ucode_need','uname','age','sex','area','degree','grade']
user_insider.ucode_need.replace('a','',inplace=True,regex=True)
user_insider['ucode_need'] = user_insider.ucode_need.map(lambda x:str(int(x)).strip())

In [59]:
cust_all.columns = ['投保单号','year_pay','year_protect','tb_gender','tb_age','bb_gender','bb_age']
col = ['ucode_need', 'age', 'sex', 'area', 'degree', 'grade']
order_off_mid = pd.merge(order_life_20,user_insider[col],how='left',on='ucode_need')
order_off_mid_01 = pd.merge(order_off_mid,cust_all,how='left',on='投保单号')
order_off_sum = pd.merge(order_off_mid_01,order_off_tb,how='left',on='投保单号')

### 线上用户客户年龄

In [60]:
order_20.ucode.fillna(0,inplace=True)
order_20['ucode_need'] = order_20.ucode.map(lambda x:str(int(x)).strip())
user_order_all = order_20.groupby(['ucode_need']).bf.sum().reset_index()
user_order_need = order_20.loc[(order_20.plabel_term ==1) & (order_20.plabel_L1 =='重疾险')
                              & (order_20.ucode_need !='0')].groupby(['ucode_need','mon']).bf.sum().reset_index()

In [61]:
order_20['bb_age'] = order_20.bb_id.map(lambda x:getAge(x))
order_20['bb_gender'] = order_20.bb_id.map(lambda x:getGender(x))
order_20['tb_age'] = order_20.tb_id.map(lambda x:getAge(x))
order_20['tb_gender'] = order_20.tb_id.map(lambda x:getGender(x))

In [64]:
col = ['ucode_need', 'age', 'sex', 'area', 'degree', 'grade']
order_online_sum = pd.merge(order_20,user_insider[col],how='left',on='ucode_need')
order_online_sum = order_online_sum.loc[order_online_sum.pname.isin(['安联女性特定重疾保险计划（18-40周岁）','安联女性特定重疾保险计划（41-50周岁）'])==False,:].copy()

### 保单合并

In [135]:
order_off_sum['order_type_1'] = '线下保单'
order_online_sum['order_type_1'] = '线上保单'

bins = [0,18,22,25,30,35,40,45,50,55,60,100]
order_off_sum['bins_tb_age'] =  pd.cut(order_off_sum.tb_age,bins =bins)
order_off_sum['bins_bb_age'] =  pd.cut(order_off_sum.bb_age,bins =bins)
order_online_sum['bins_tb_age'] =  pd.cut(order_online_sum.tb_age,bins =bins)
order_online_sum['bins_bb_age'] =  pd.cut(order_online_sum.bb_age,bins =bins)

order_online_sum.loc[order_online_sum.plabel_need.isin(['重疾险_其他','重疾险_少儿']),'order_type_2'] = '线上重疾'
order_off_sum.loc[order_off_sum.pname.str.contains('重大疾病'),'order_type_2'] = '线下重疾'

col = ['ucode_need','age','sex','area','degree','grade','pname',
       'tb_name','tb_age','tb_gender','bb_name','bb_age','bb_gender',
      'bins_tb_age','bins_bb_age','order_type_1','order_type_2']

In [136]:
order_all = pd.concat([order_online_sum[col],order_off_sum[col]])

In [137]:
## 全产品投保人分布
order_all.groupby(['order_type_1','bins_tb_age']).tb_name.nunique().reset_index().pivot('bins_tb_age','order_type_1','tb_name')

order_type_1,线上保单,线下保单
bins_tb_age,,
"(0, 18]",164,35
"(18, 22]",7995,552
"(22, 25]",10991,1376
"(25, 30]",35072,7169
"(30, 35]",56520,10135
"(35, 40]",47425,7624
"(40, 45]",38531,4675
"(45, 50]",40006,3793
"(50, 55]",34518,1865


In [138]:
## 重疾产品投保人分布
order_all.groupby(['order_type_2','bins_tb_age']).tb_name.nunique().reset_index().pivot('bins_tb_age','order_type_2','tb_name')

order_type_2,线上重疾,线下重疾
bins_tb_age,,
"(0, 18]",3,23
"(18, 22]",104,417
"(22, 25]",263,1139
"(25, 30]",2359,5987
"(30, 35]",4107,8079
"(35, 40]",2403,5632
"(40, 45]",797,3324
"(45, 50]",295,2547
"(50, 55]",96,997


In [132]:
## 全产品被保人分布
order_all.groupby(['order_type_1','bins_bb_age']).tb_name.nunique().reset_index().pivot('bins_bb_age','order_type_1','tb_name')

order_type_1,线上保单,线下保单
bins_bb_age,,
"(0, 18]",53645,10457
"(18, 22]",11615,1509
"(22, 25]",11681,1418
"(25, 30]",29328,5063
"(30, 35]",43051,7020
"(35, 40]",37150,5454
"(40, 45]",32857,3374
"(45, 50]",38161,3013
"(50, 55]",37763,1635


In [139]:
## 重疾被保人分布
order_all.groupby(['order_type_2','bins_bb_age']).bb_name.nunique().reset_index().pivot('bins_bb_age','order_type_2','bb_name')

order_type_2,线上重疾,线下重疾
bins_bb_age,,
"(0, 18]",7714,8906
"(18, 22]",117,1117
"(22, 25]",167,1104
"(25, 30]",947,4204
"(30, 35]",1391,5835
"(35, 40]",869,4324
"(40, 45]",295,2565
"(45, 50]",148,2153
"(50, 55]",100,936


In [151]:
order_all.loc[order_all.tb_gender !='未知',:].groupby(['order_type_1','tb_gender']).tb_name.nunique().reset_index().\
pivot('order_type_1','tb_gender','tb_name')

tb_gender,女,男
order_type_1,,
线上保单,135570,149710
线下保单,22655,12614


In [152]:
order_all.loc[order_all.tb_gender !='未知',:].groupby(['order_type_2','tb_gender']).tb_name.nunique().reset_index().\
pivot('order_type_2','tb_gender','tb_name')

tb_gender,女,男
order_type_2,,
线上重疾,6311,3684
线下重疾,16617,9841


In [154]:
order_all.loc[order_all.bb_gender !='未知',:].groupby(['order_type_1','bb_gender']).bb_name.nunique().reset_index().\
pivot('order_type_1','bb_gender','bb_name')

bb_gender,女,男
order_type_1,,
线上保单,157047,201831
线下保单,23646,20740


In [155]:
order_all.loc[order_all.bb_gender !='未知',:].groupby(['order_type_2','bb_gender']).bb_name.nunique().reset_index().\
pivot('order_type_2','bb_gender','bb_name')

bb_gender,女,男
order_type_2,,
线上重疾,6099,6269
线下重疾,17509,15769


###  客户年龄情况

In [34]:
bins = [0,18,22,25,30,35,40,45,50,55,60,100]
order_off_sum['bins_cust_age'] =  pd.cut(order_off_sum.bb_age,bins =bins)
order_off_sum['bins_tb_age'] =  pd.cut(order_off_sum.tb_age,bins =bins)

In [25]:
order_off_sum.groupby(['bins_cust_age','bb_gender']).bb_name.nunique().reset_index().pivot(index='bins_cust_age',columns = 'bb_gender',values='bb_name')

bb_gender,女,男
bins_cust_age,,
"(0, 18]",5823,6679
"(18, 22]",743,760
"(22, 25]",815,610
"(25, 30]",3095,2089
"(30, 35]",4257,3055
"(35, 40]",3230,2499
"(40, 45]",1961,1564
"(45, 50]",1826,1355
"(50, 55]",998,717


In [36]:
order_off_sum.groupby(['bins_tb_age','tb_gender']).tb_name.nunique().reset_index().pivot(index='bins_tb_age',columns = 'tb_gender',values='tb_name')

tb_gender,女,男
bins_tb_age,,
"(0, 18]",17,18
"(18, 22]",322,230
"(22, 25]",907,470
"(25, 30]",4767,2468
"(30, 35]",6638,3650
"(35, 40]",4981,2730
"(40, 45]",3075,1627
"(45, 50]",2531,1276
"(50, 55]",1225,645


In [74]:
order_online_sum['bins_cust_age'] =  pd.cut(order_online_sum.bb_age,bins =bins)
order_online_sum.groupby(['bb_gender','bins_cust_age']).bb_name.nunique().reset_index().\
pivot(index='bins_cust_age',columns ='bb_gender',values='bb_name')

bb_gender,女,男
bins_cust_age,,
"(0, 18]",3368.0,3743.0
"(18, 22]",43.0,57.0
"(22, 25]",80.0,56.0
"(25, 30]",452.0,341.0
"(30, 35]",637.0,535.0
"(35, 40]",378.0,361.0
"(40, 45]",125.0,128.0
"(45, 50]",67.0,59.0
"(50, 55]",49.0,35.0


In [89]:
order_online_sum['bins_tb_age'] =  pd.cut(order_online_sum.tb_age,bins =bins)
order_online_sum['bins_cust_age'] =  pd.cut(order_online_sum.bb_age,bins =bins)
order_online_sum.groupby(['tb_gender','bins_tb_age']).tb_name.nunique().reset_index().\
pivot(index='bins_tb_age',columns ='tb_gender',values='tb_name')

tb_gender,女,男
bins_tb_age,,
"(0, 18]",73,109
"(18, 22]",4275,4801
"(22, 25]",7121,6094
"(25, 30]",25466,17211
"(30, 35]",42683,26777
"(35, 40]",35974,22634
"(40, 45]",27683,19283
"(45, 50]",25509,21332
"(50, 55]",16191,19079


In [76]:
order_online_sum.groupby('plabel_need').order.count().reset_index()

,plabel_need,order
0,重疾险_其他,4313
1,重疾险_少儿,7689


In [56]:
order_online_sum.groupby(['bb_gender','bins_cust_age']).bb_name.nunique().reset_index().pivot(index='bins_cust_age',columns ='bb_gender',values='bb_name')

bb_gender,女,男
bins_cust_age,,
"(0, 18]",31838,38736
"(18, 22]",5991,6781
"(22, 25]",7290,6659
"(25, 30]",21551,16404
"(30, 35]",34155,24719
"(35, 40]",29075,21753
"(40, 45]",23619,19164
"(45, 50]",23995,22383
"(50, 55]",17905,21932


In [86]:
order_online_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12002 entries, 0 to 593553
Data columns (total 92 columns):
order               12002 non-null object
order_origin        12002 non-null object
order_stat          12002 non-null object
time_order          12002 non-null object
time_pay            12002 non-null object
time_cb             12002 non-null object
time_start          12002 non-null object
time_end            0 non-null object
pcode               12002 non-null object
pid                 12002 non-null object
pname               12002 non-null object
pcompany            12002 non-null object
plan                12002 non-null object
pay_way             360 non-null object
channel_src         12002 non-null object
channel_detail      12002 non-null object
batch_id            0 non-null float64
order_renew_src     0 non-null object
is_self             6855 non-null object
bf                  12002 non-null float64
tgf_rate_A          12002 non-null object
tgf_rate_B          1

In [80]:
order_online_sum.groupby(['plabel_need','relation']).order.count().reset_index().pivot(index='plabel_need',columns ='relation',values='order')

relation,丈夫,儿子,其它,女儿,妻子,本人,母亲,父亲
plabel_need,,,,,,,,
重疾险_其他,357.0,336.0,2014.0,283.0,210.0,1065.0,25.0,23.0
重疾险_少儿,NaN,4041.0,NaN,3648.0,NaN,NaN,NaN,NaN


In [78]:
order_off_sum.loc[order_off_sum.bb_age <=18,:].groupby('pname').bb_name.nunique().reset_index().sort_values('bb_name',ascending=False)

,pname,bb_name
125,恒大恒家保终身重大疾病保险,3514
36,信泰完美人生守护尊享版重大疾病保险,2430
133,横琴琴童尊享增额终身寿险,1512
52,光大永明童佳保尊享版重大疾病保险,1230
93,天安人寿健康源2019增强版终身重大疾病保险,947
149,百年医惠通费用补偿医疗保险,866
63,华夏华夏红钻石版年金保险,842
61,华夏华夏福多倍2.0版重大疾病保险,838
153,百年康盛保终身重大疾病保险,651
39,信泰百万守护2019重大疾病保险,621


In [61]:
order_online_sum.plabel_need.unique()

array(['重疾险_少儿', '健康险_其他', '重疾险_其他', '定寿', '健康险_百万医疗', '意外险_其他', '终身寿',
       '意外险_旅游险', '健康险_住院保', '意外险_10+1', '意外险_学平险', '财产险', '服务类', nan],
      dtype=object)

In [63]:
order_online_sum.loc[order_online_sum.plabel_need.isin(['意外险_旅游险','意外险_学平险'])==False,:].groupby(['tb_gender','bins_tb_age']).tb_name.nunique().reset_index().pivot(index='bins_tb_age',columns ='tb_gender',values='tb_name')

tb_gender,女,男
bins_tb_age,,
"(0, 18]",59,95
"(18, 22]",3167,3904
"(22, 25]",6161,5147
"(25, 30]",23230,15252
"(30, 35]",38776,23989
"(35, 40]",32354,20096
"(40, 45]",24815,17227
"(45, 50]",22870,19371
"(50, 55]",13683,17469


In [66]:
order_online_sum.columns

Index(['order', 'order_origin', 'order_stat', 'time_order', 'time_pay',
       'time_cb', 'time_start', 'time_end', 'pcode', 'pid', 'pname',
       'pcompany', 'plan', 'pay_way', 'channel_src', 'channel_detail',
       'batch_id', 'order_renew_src', 'is_self', 'bf', 'tgf_rate_A',
       'tgf_rate_B', 'tgf_A', 'tgf_B', 'tgf_tax', 'tjf_tax', 'tjf', 'tgf_b1',
       'tgf_b2', 'money_add', 'ticket_code', 'ticket_name', 'ticket_rate',
       'money_top', 'ticket_value', 'tgf_X', 'uid', 'order_origin_uid',
       'uname', 'ucell', 'uprov', 'ucity', 'uarea', 'is_insider', 'ucode',
       'time_join', 'area_type', 'area_work', 'area_service_id',
       'area_service', 'team_sale_id', 'team_sale_name', 'team_bus_id',
       'team_bus_name', 'tj_name', 'tj_uid', 'tj_cell', 'tj_prov', 'tj_city',
       'tj_area', 'tj_area_work', 'tj_area_service', 'tb_name', 'tb_id_type',
       'tb_id', 'tb_email', 'tb_cell', 'relation', 'bb_name', 'bb_id_type',
       'bb_id', 'date_pay', 'mon', 'plabel_L1', 'p

In [68]:
order_online_sum.loc[(order_online_sum.tb_age >=50)
                     ,:].groupby('relation').tb_name.nunique().reset_index().sort_values('tb_name',ascending=False)

,relation,tb_name
7,本人,94808
1,儿子,3085
5,女儿,2507
0,丈夫,2506
6,妻子,1350
8,母亲,840
9,父亲,420
2,其他,76
3,其它,57
4,劳动关系,2


### 用户画像梳理

In [207]:
bins = [0,18,22,25,30,35,40,45,50,55,60,100]
order_off_sum['bins_uid_age'] =  pd.cut(order_off_sum.age,bins =bins)

In [214]:
order_off_sum.columns

Index(['序号', '保单状态', 'date_tb', '回单日期', '保单录入日期', '承保日期', '预收日期', 'date_input',
       '销售团队代码', '销售团队名称', '地区分类', '团队分类', '是否本部', '营业部代码', '营业部名称', '是否营业中心',
       '营业部所属批牌机构编码', '营业部所属批牌', '营业部类型', '代理人代码', '代理人姓名', '入司日期', '代理人职级',
       '业务合伙人级别', '经营合伙人级别', '经营团队总经理标志', '职业化标志', '推荐人代码', '推荐人姓名', '辅导人代码',
       '辅导人姓名', '经营合伙人代码', '经营合伙人姓名', '离职日期', 'ucode', '现代理人姓名', 'area_type',
       '原辅代理人姓名', '原辅代理人编码', '原辅代理人所在销售团队名称', '原辅代理人所在销售团队代码', '原辅代理人所在机构',
       '现辅代理人姓名', '现辅代理人编码', '现辅代理人所在销售团队名称', '现辅代理人所在销售团队代码', '是否自保件',
       '保险公司代码', '保险公司名称', '投保单号', '保单号', 'tb_name', 'bb_name', '险种代码',
       '外部险种代码', 'pname', '是否主险', '产品分类', '交费年期|年龄标志', '缴费期', '佣金率', '规模保费',
       'bf', '保额', '保全标志', '是否纳入现代理人方案统计', '归属类型', '归属细类', '保单关系人', '代码', '姓名',
       '月份', '销服代码', '销服名称', '机构', '职级', '入司日期.1', '入司月', '营业部代码.1', '营业部名称.1',
       '是否营业中心.1', '师傅代码', '师傅姓名', 'ucode_need', 'mon', 'age', 'sex', 'area',
       'degree', 'grade', 'bins_uid_age'],
      dtype='object')

In [216]:
order_off_sum.groupby(['bins_uid_age','sex']).ucode_need.nunique().reset_index().pivot(index='bins_uid_age',columns ='sex',values='ucode_need')

sex,1,2
bins_uid_age,,
"(18, 22]",23,64
"(22, 25]",95,128
"(25, 30]",409,764
"(30, 35]",912,2220
"(35, 40]",828,2393
"(40, 45]",620,1545
"(45, 50]",487,1073
"(50, 55]",265,691
"(55, 60]",102,329


In [210]:
order_online_sum['bins_uid_age'] =  pd.cut(order_online_sum.age,bins =bins)

In [222]:
order_online_sum.groupby(['bins_uid_age','sex']).ucode_need.nunique().reset_index().pivot(index='bins_uid_age',columns ='sex',values='ucode_need')

sex,1.0,2.0
bins_uid_age,,
"(18, 22]",6,13
"(22, 25]",25,38
"(25, 30]",192,353
"(30, 35]",443,1064
"(35, 40]",367,1013
"(40, 45]",194,510
"(45, 50]",106,249
"(50, 55]",50,140
"(55, 60]",23,85


In [228]:
order_online_sum.groupby(['sex','degree']).ucode_need.nunique().reset_index().pivot(index='sex',columns ='degree',values='ucode_need')

degree,专科,中专,其他,初中,博士,小学,本科,硕士,高中
sex,,,,,,,,,
1.0,432.0,88.0,NaN,20.0,1.0,2.0,653.0,80.0,144.0
2.0,1082.0,215.0,1.0,77.0,10.0,5.0,1534.0,267.0,310.0


In [229]:
order_off_sum.groupby(['sex','degree']).ucode_need.nunique().reset_index().pivot(index='sex',columns ='degree',values='ucode_need')

degree,专科,中专,其他,初中,博士,小学,本科,硕士,高中
sex,,,,,,,,,
1,1149.0,355.0,NaN,130.0,8.0,8.0,1340.0,176.0,627.0
2,2897.0,870.0,1.0,381.0,18.0,40.0,3096.0,476.0,1553.0


In [236]:
order_off_sum.groupby('area').ucode_need.nunique().reset_index()

,area,ucode_need
0,上海,295
1,云南,319
2,其他,7
3,内蒙,174
4,北京,1247
5,厦门,130
6,四川,1876
7,大连,246
8,天津,439
9,安徽,150


In [237]:
order_online_sum.groupby('area').ucode_need.nunique().reset_index()

,area,ucode_need
0,上海,176
1,云南,105
2,其他,34
3,内蒙,70
4,北京,568
5,厦门,76
6,四川,758
7,大连,109
8,天津,142
9,安徽,75


In [239]:
res = pd.merge(order_online_sum,order_off_sum[['ucode_need','bf']].drop_duplicates(),how='left',on='ucode_need')

In [242]:
res.loc[res.bf_y.notna(),:].groupby('bins_uid_age').ucode_need.nunique().reset_index()

,bins_uid_age,ucode_need
0,"(0, 18]",0
1,"(18, 22]",11
2,"(22, 25]",29
3,"(25, 30]",336
4,"(30, 35]",1082
5,"(35, 40]",1066
6,"(40, 45]",552
7,"(45, 50]",276
8,"(50, 55]",138
9,"(55, 60]",70


In [244]:
res.loc[res.bf_y.notna(),:].groupby(['bins_uid_age','sex']).ucode_need.nunique().reset_index().pivot(index='bins_uid_age',columns ='sex',values='ucode_need')

sex,1.0,2.0
bins_uid_age,,
"(18, 22]",3,8
"(22, 25]",16,13
"(25, 30]",109,227
"(30, 35]",303,779
"(35, 40]",241,825
"(40, 45]",138,414
"(45, 50]",76,200
"(50, 55]",35,103
"(55, 60]",14,56


In [245]:
res.loc[res.bf_y.notna(),:].groupby(['degree']).ucode_need.nunique().reset_index()
# .pivot(index='bins_uid_age',columns ='sex',values='ucode_need')

,degree,ucode_need
0,专科,1086
1,中专,225
2,其他,1
3,初中,56
4,博士,9
5,小学,4
6,本科,1625
7,硕士,266
8,高中,316


In [246]:
res.loc[res.bf_y.notna(),:].groupby(['area']).ucode_need.nunique().reset_index()

,area,ucode_need
0,上海,138
1,云南,81
2,其他,1
3,内蒙,48
4,北京,398
5,厦门,60
6,四川,497
7,大连,77
8,天津,117
9,安徽,40


In [ ]:
res.loc[res]

### 线上重疾与线下保单交叉情况

In [84]:
agg_col = {'ucode_need':'nunique','bf':'sum'}

In [20]:
order_20.ucode.fillna(0,inplace=True)
order_20['ucode_need'] = order_20.ucode.map(lambda x:str(int(x)).strip())
user_order_all = order_20.groupby(['ucode_need']).bf.sum().reset_index()
user_order_need = order_20.loc[(order_20.plabel_term ==1) & (order_20.plabel_L1 =='重疾险')
                              & (order_20.ucode_need !='0')].groupby(['ucode_need','mon']).bf.sum().reset_index()

In [81]:
user_order_need.groupby('mon').agg(agg_col)

,ucode_need,bf
mon,,
2020-01,1634,6477055.25
2020-02,1337,4890671.80
2020-03,1454,4956509.04
2020-04,1622,6071477.77
2020-05,1169,3791532.57


In [82]:
user_order_need.agg(agg_col)

ucode_need        4822.00
bf            26187246.43
dtype: float64

In [83]:

user_life_all = order_life_20.loc[order_life_20.ucode_need != '0',:].groupby(['ucode_need','mon']).bf.sum().reset_index()

In [99]:
order_life_20['order_type'] = np.nan
order_life_20.loc[order_life_20.pname.str.contains('重大疾病'),'order_type'] ='线下重疾'
order_life_20.order_type.fillna('其他',inplace=True)

In [109]:
order_life_20['bf_std'] = np.nan
order_life_20.loc[order_life_20.order_type == '线下重疾','bf_std'] =order_life_20.loc[order_life_20.order_type == '线下重疾','bf']
order_life_20.bf_std.fillna(0,inplace=True)
order_life_20['ucode_zj'] = np.nan
order_life_20.loc[order_life_20.order_type == '线下重疾','ucode_zj'] =order_life_20.loc[order_life_20.order_type == '线下重疾','ucode_need']

In [115]:
user_life_need = order_life_20.loc[order_life_20.ucode_zj.notna(),:].groupby(['ucode_need','mon']).bf_std.sum().reset_index()

In [110]:
agg_col = {'ucode_need':'nunique','bf':'sum','ucode_zj':'nunique','bf_std':'sum'}
order_life_20.groupby('mon').agg(agg_col)

,ucode_need,bf,ucode_zj,bf_std
mon,,,,
2020-01,6468,1.180469e+08,5229,63966194.28
2020-02,3451,4.171721e+07,2773,28255768.85
2020-03,5544,8.049735e+07,4303,44974053.32
2020-04,5660,8.784301e+07,4451,49895997.33
2020-05,3316,4.825155e+07,2530,25446042.58


In [112]:
order_life_20.agg(agg_col).reset_index()

,index,0
0,ucode_need,1.293500e+04
1,bf,3.763560e+08
2,ucode_zj,1.105900e+04
3,bf_std,2.125381e+08


In [130]:
agg_col = {'ucode_need':'nunique','bf_x':'sum','bf_y':'sum'}
user_cross = pd.merge(user_order_need,user_life_all,how='left',on=['ucode_need','mon'])
user_cross.loc[user_cross.bf_y.notna(),:].groupby('mon').agg(agg_col).reset_index()

,mon,ucode_need,bf_x,bf_y
0,2020-01,950,3462070.68,23849727.35
1,2020-02,616,2111920.40,10184028.95
2,2020-03,820,2644792.94,16082094.36
3,2020-04,932,3471655.97,20185525.99
4,2020-05,483,1526387.43,7878557.25


In [131]:
user_cross = pd.merge(user_order_need,user_life_all,how='left',on=['ucode_need'])
user_cross.loc[user_cross.bf_y.notna(),:].agg(agg_col).reset_index()

,index,0
0,ucode_need,3.495000e+03
1,bf_x,5.262326e+07
2,bf_y,3.179466e+08


In [132]:
agg_col = {'ucode_need':'nunique','bf':'sum','bf_std':'sum'}
user_cross_zj = pd.merge(user_order_need,user_life_need,how='left',on=['ucode_need','mon'])
user_cross_zj.loc[user_cross_zj.bf_std.notna(),:].groupby('mon').agg(agg_col).reset_index()

,mon,ucode_need,bf,bf_std
0,2020-01,851,3036761.68,13263341.34
1,2020-02,549,1833220.00,7088260.87
2,2020-03,720,2257879.04,9905311.08
3,2020-04,830,3039139.27,11658622.04
4,2020-05,411,1267363.41,4484438.99


In [137]:
user_cross_zj = pd.merge(user_order_need,user_life_need,how='left',on=['ucode_need'])

user_cross_zj.loc[user_cross_zj.bf_std.notna(),:].agg(agg_col).reset_index()

,index,0
0,ucode_need,3.218000e+03
1,bf,4.443879e+07
2,bf_std,1.801151e+08


In [128]:
order_life_20.loc[order_life_20.order_type=='线下重疾'].groupby('pname').bf.sum().reset_index().sort_values('bf',ascending=False)

,pname,bf
73,恒大恒家保终身重大疾病保险,59017273.93
20,信泰完美人生守护尊享版重大疾病保险,30542916.20
31,光大永明童佳保尊享版重大疾病保险,18394223.67
58,天安人寿健康源2019增强版终身重大疾病保险,17415753.91
38,华夏华夏福多倍2.0版重大疾病保险,12462925.16
24,信泰百万守护2019重大疾病保险,12094898.97
99,百年康多保终身重大疾病保险,6788484.25
42,同方全球康健一生新多倍保终身重大疾病保险,5734956.75
101,百年康盛保终身重大疾病保险,5599934.05
21,信泰完美人生守护重大疾病保险,5427862.09


## 旧日报

### 基本处理

In [94]:
## 区域中心修正
order_sum.loc[(order_sum.area_type =='大童直属区域中心') 
              | (order_sum.area_type.isna()),'area_type'] = '其他'

order_2019_need.loc[(order_2019_need.area_type =='大童直属区域中心') 
              | (order_2019_need.area_type.isna()),'area_type'] = '其他'

NameError: name 'order_sum' is not defined

In [88]:
agg_col_01 = {'uid':'nunique','order':'count'}
agg_col_02 = {'bf':'sum'}

In [46]:
now = datetime.datetime.now()-datetime.timedelta(hours=24)
# now =pd.to_datetime('2019-01-25')
curmon = '2020-04'
last_mon = '2020-03'
curmon_sdate = mon_sdate_map[curmon]
lastmon_sdate = mon_sdate_map[last_mon]
curmon_edate = pd.to_datetime(now.strftime('%Y-%m-%d'))
days_delta = curmon_edate -curmon_sdate
lastmon_edate = lastmon_sdate + days_delta
print('当前数据时间%s'%now)
print('当月开始时间%s,当月结束时间%s'%(curmon_sdate,curmon_edate))
print('上月开始时间%s,上月结束时间%s'%(lastmon_sdate,lastmon_edate))

ysdate = pd.to_datetime('2018-12-26')
yedate = pd.to_datetime(pd.to_datetime(pd.Series(ysdate + (now - pd.to_datetime('2019-12-26'))).dt.strftime('%Y-%m-%d')).unique()[0])
print('去年结束时间%s'%(yedate))

model =2
## 需要修改 
start = pd.to_datetime('2020-04-17')
end = pd.to_datetime('2020-04-19')


tsdate = day_date_descision(model,start,end)[0]
tedate = day_date_descision(model,start,end)[1]

print('当日数据时间%s'%tsdate)

当前数据时间2020-04-19 10:11:31.604645
当月开始时间2020-03-26 00:00:00,当月结束时间2020-04-19 00:00:00
上月开始时间2020-02-26 00:00:00,上月结束时间2020-03-21 00:00:00
去年结束时间2019-04-20 00:00:00
当日数据时间2020-04-17 00:00:00


### 目标数据

In [47]:
area_target = pd.read_excel('D:\\User\\Desktop\\dic_info\\01_2020年各机构月度目标表_new.xlsx')
area_target = area_target[['area_name',curmon,'target_year']].copy()
area_target.columns = ['area_name','mon_target','year_target']

### 对比数据

In [48]:
order_new_compare_mon = order_2019_need.loc[(order_2019_need.order.isin(order_renew_list)==False)
                   & (order_2019_need.date_pay >= lastmon_sdate)
                   & (order_2019_need.date_pay <= lastmon_edate),:].groupby('area_type').bf.sum().reset_index()
order_new_compare_year = order_2019_need.loc[(order_2019_need.order.isin(order_renew_list)==False)
                   & (order_2019_need.date_pay >= ysdate)
                   & (order_2019_need.date_pay <= yedate),:].groupby('area_type').bf.sum().reset_index()

In [49]:
order_new_compare = pd.merge(order_new_compare_mon,order_new_compare_year,how='left',on='area_type',suffixes=['_compare_mon','_compare_year'])
order_new_compare.fillna(0,inplace=True)
order_new_compare['bf_compare_mon'] = order_new_compare.bf_compare_mon/10000
order_new_compare['bf_compare_year'] = order_new_compare.bf_compare_year/10000
del order_new_compare_mon
del order_new_compare_year

### 日报表数据

In [50]:
## 日数据
order_day_uid  = order_sum.loc[(order_sum.date_pay>=tsdate) & (order_sum.date_pay<=tedate),:].groupby(['area_type']).agg(agg_col_01).reset_index()
order_day_bf =   order_sum.loc[(order_sum.date_pay>=tsdate) & (order_sum.date_pay<=tedate),:].groupby(['area_type','is_renew']).agg(agg_col_02).reset_index().pivot(index='area_type',columns ='is_renew',values='bf').reset_index()

In [51]:
## 月数据
order_mon_uid  = order_sum.loc[order_sum.mon ==curmon,:].groupby(['area_type']).agg(agg_col_01).reset_index()
order_mon_bf =   order_sum.loc[order_sum.mon ==curmon,:].groupby(['area_type','is_renew']).agg(agg_col_02).reset_index().pivot(index='area_type',columns ='is_renew',values='bf').reset_index()

In [52]:
## 年数据
order_year_uid  = order_sum.loc[(order_sum.date_pay <= pd.to_datetime('2020-12-25')) & (order_sum.date_pay>=pd.to_datetime('2019-12-26'))].groupby(['area_type']).agg(agg_col_01).reset_index()
order_year_bf   = order_sum.loc[(order_sum.date_pay <= pd.to_datetime('2020-12-25')) & (order_sum.date_pay>=pd.to_datetime('2019-12-26'))].groupby(['area_type','is_renew']).agg(agg_col_02).reset_index().pivot(index='area_type',columns ='is_renew',values='bf').reset_index()

In [53]:
order_d = pd.merge(order_day_uid,order_day_bf,how='left',on='area_type')
order_m = pd.merge(order_mon_uid,order_mon_bf,how='left',on='area_type')
order_y = pd.merge(order_year_uid,order_year_bf,how='left',on='area_type')

In [54]:
res_mid = pd.merge(order_y,order_m,how='left',on='area_type')
res_final = pd.merge(res_mid,order_d,how='left',on='area_type')

In [55]:
res_final.rename(columns={'0_x':'0','1_x':'1','0':'0_x','1':'1_x','uid':'uid_x','order':'order_x','uid_x':'uid','order_x':'order'},inplace=True)

In [56]:
res_area = pd.merge(res_final,area_name,how='left',on='area_type')
res_target = pd.merge(res_area,area_target,how='left',on='area_name')
res_compare = pd.merge(res_target,order_new_compare,how='left',on='area_type')

In [57]:
res_compare.fillna(0,inplace=True)
res_compare['0_x'] = res_compare['0_x']/10000
res_compare['1_x'] = res_compare['1_x']/10000
res_compare['0_y'] = res_compare['0_y']/10000
res_compare['1_y'] = res_compare['1_y']/10000
res_compare['0']   = res_compare['0']/10000
res_compare['1']   = res_compare['1']/10000

In [58]:
res_compare['day_sum'] =  res_compare['0_x'] + res_compare['1_x']
res_compare['mon_sum'] =  res_compare['0_y'] + res_compare['1_y']
res_compare['year_sum'] = res_compare['0']   + res_compare['1']

In [59]:
res_compare['mon_finish_rate'] = res_compare.mon_sum/res_compare.mon_target
res_compare['year_finish_rate'] = res_compare.year_sum/res_compare.year_target
res_compare['bf_mon_reduce'] = res_compare['0_y'] - res_compare.bf_compare_mon
res_compare['bf_year_reduce'] = res_compare['0'] - res_compare.bf_compare_year

In [60]:
need_col = ['area_name', 'uid_x', 'order_x', '0_x', '1_x','day_sum', 'uid_y', 'order_y',
       '0_y', 'bf_mon_reduce','1_y','mon_sum','mon_target','mon_finish_rate','uid', 'order','0','bf_year_reduce', '1','year_sum','year_target','year_finish_rate']

In [62]:
report_01 = res_compare.loc[res_compare.area_name !='其他',need_col].sort_values('mon_sum',ascending=False)
report_02 = res_compare.loc[res_compare.area_name =='其他',need_col]
report = pd.concat([report_01,report_02])
report.replace(np.inf,0,inplace=True)
report.reset_index().to_excel('D:\\User\\Desktop\\周期数据\\日报\\001_数据_网销业务考核表.xlsx')

## 交服总奖励方案日报

In [28]:
mreport = pd.read_excel(dir_down + '交服网销业务追踪 (16).xlsx',header=2)
qreport = pd.read_excel(dir_down + '交服网销业务追踪 (17).xlsx',header=2)
yreport = pd.read_excel(dir_down + '交服网销业务追踪 (18).xlsx',header=2)

In [29]:
mreport.head()

,业绩排名,交服名称,区域中心名称,保费
0,NaN,合计,NaN,14378997.71
1,1.0,四川高端,四川,1270940.52
2,2.0,深圳高端,深圳,811185.84
3,3.0,北京恒基,北京,666368.10
4,4.0,北京朝阳,北京,657617.97


In [30]:
col = ['交服名称','保费']
res_mid = pd.merge(yreport,qreport[col],how='left',on='交服名称',suffixes=['_y','_q'])
res = pd.merge(res_mid,mreport[col],how='left',on='交服名称')

In [31]:
res_need = res.loc[res.区域中心名称.notna(),:].sort_values('保费',ascending=False)
res_need['保费'] = res_need.保费.map(lambda x:round(x/10000,2))
res_need['保费_q'] = res_need.保费_q.map(lambda x:round(x/10000,2))
res_need['保费_y'] = res_need.保费_y.map(lambda x:round(x/10000,2))

In [32]:
bins = [0,20,30,50,100,999]
labels= ['','2000元','3000元','5000元','10000元']
res_need['奖励'] = pd.cut(res_need.保费,bins = bins,right=False,labels=labels)
col_need = ['交服名称','区域中心名称','保费','奖励','保费_q','保费_y']
res_need[col_need].to_excel('D:\\User\\Desktop\\周期数据\\交服奖励方案日报-数据.xlsx',index=0)

## 业务类型细分表

In [ ]:
order_renew = pd.read_excel('E:\\SynologyDrive\\dir_dic\\10_order_renew_new.xlsx')

In [9]:
agg_col = {'uid':'nunique','bf':'sum','order':'count'}

In [4]:
order_all = pd.read_csv('E:\\SynologyDrive\\dir_order\\order_report_0428.csv',encoding='utf-8',parse_dates=['date_pay'])
order_sum = order_all.loc[order_all.order_stat.isin(['待生效','已生效','已发货'])].copy()
order_sum['mon'] = order_sum.date_pay.map(date_bus)

In [5]:
order_ptype = pd.merge(order_sum,ptype,how='left',on='pname')
order_net = order_ptype.copy()

In [7]:
order_net.loc[order_net.plabel_term ==1,'is_term'] = '2_期缴产品'
order_net.is_term.fillna('1_一年期产品',inplace=True)

In [8]:
order_net['is_insider_new'] = order_net.is_insider
order_net.loc[order_net.uid == 'U00000000','is_insider_new'] = '3_官方'
order_net.loc[order_net.is_insider_new =='是','is_insider_new'] = '1_司内'
order_net.loc[order_net.is_insider_new =='否','is_insider_new'] = '2_司外'

In [10]:
order_net.groupby(['mon','is_term','is_insider_new']).agg(agg_col).reset_index().to_excel('12.xlsx',encoding='gbk',index=0)

In [11]:
order_net.groupby(['mon','is_term']).agg(agg_col).reset_index().tail(6)

,mon,is_term,uid,bf,order
28,2020-03,1_一年期产品,20652,30507036.80,91564
29,2020-03,2_期缴产品,2249,7591232.05,4000
30,2020-04,1_一年期产品,22488,34327720.66,111275
31,2020-04,2_期缴产品,2455,9196792.53,4507
32,2020-05,1_一年期产品,4195,3126433.74,9907
33,2020-05,2_期缴产品,327,856718.60,434


In [12]:
order_net.groupby(['mon','is_insider_new']).agg(agg_col).reset_index().tail(6)

,mon,is_insider_new,uid,bf,order
45,2020-04,1_司内,13467,32915342.84,75628
46,2020-04,2_司外,9608,10487253.19,39411
47,2020-04,3_官方,1,121917.16,743
48,2020-05,1_司内,2971,3128636.33,6973
49,2020-05,2_司外,1400,842201.01,3294
50,2020-05,3_官方,1,12315.00,74


In [13]:
order_net.groupby(['mon']).agg(agg_col).reset_index().tail()

,mon,uid,bf,order
12,2020-01,21920,37939980.10,115395
13,2020-02,21212,38722899.55,115640
14,2020-03,21174,38098268.85,95564
15,2020-04,23048,43524513.19,115782
16,2020-05,4372,3983152.34,10341


In [82]:
order_net.groupby(['mon','is_term','is_insider_new']).agg(agg_col).reset_index().tail(8)

,mon,is_term,is_insider_new,uid,bf,order
29,2019-05,2_期缴产品,3_官方,1,3487.00,5
30,2019-06,1_一年期产品,1_司内,14325,19682763.27,114862
31,2019-06,1_一年期产品,2_司外,9983,7870291.88,63573
32,2019-06,1_一年期产品,3_官方,1,244514.68,3354
33,2019-06,2_期缴产品,1_司内,2880,7085538.15,5548
34,2019-06,2_期缴产品,2_司外,369,546327.94,538
35,2019-06,2_期缴产品,3_官方,1,13154.00,11
36,2019-07,1_一年期产品,1_司内,1,199.00,1


## 撤退保

In [33]:
ct_order = pd.read_excel(dir_order + 'ct_order_0426.xlsx')

In [34]:
order_sum['is_ct'] = np.nan
order_sum.loc[order_sum.order.isin(ct_order.order.unique()),'is_ct'] = '撤退'
order_sum.is_ct.fillna('未撤退',inplace=True)

In [56]:
col = ['order','date_pay','bb_id','pname','bf','tgf_A','tgf_B','tgf_X','plan']
order_ct = order_sum.loc[order_sum.is_ct =='撤退',col].copy()
order_nct = order_sum.loc[order_sum.is_ct !='撤退',col].copy()

In [59]:
res = pd.merge(order_ct,order_nct[['bb_id','date_pay','pname','bf','plan']],how='left',on='bb_id')

In [72]:
res.loc[res.bf_y.notna(),['order','bf_x','pname_x','plan_x','bf_y','pname_y','plan_y']]

,order,bf_x,pname_x,plan_x,bf_y,pname_y,plan_y
0,20191229220030256,728.0,和谐健康慧馨安少儿定期重大疾病保险,20年,60.00,平安少儿意外伤害保障计划(升级版),基础版
1,20191229220030256,728.0,和谐健康慧馨安少儿定期重大疾病保险,20年,928.00,和谐健康慧馨安少儿定期重大疾病保险,20年
2,20191229200025927,1210.0,华贵大麦定期寿险,30年,720.00,华贵大麦正青春定期寿险,30年
3,20191229110006817,1248.0,华贵大麦定期寿险,10年,1638.20,华贵大麦2020定期寿险,10年
4,20191229110006255,778.0,华贵大麦定期寿险,10年,1042.00,华贵大麦2020定期寿险,10年
5,20191228220030202,3988.0,百年康惠保(2020版)重大疾病保险,30年,3870.00,百年康惠保(2020版)重大疾病保险,20年
6,20191228160019121,3430.0,百年康惠保(2020版)重大疾病保险,30年,486.00,众安尊享e生2019版,个人版
7,20191228140012633,185.0,华贵大麦定期寿险,10年,148.00,华贵大麦2020定期寿险,10年
9,20191227210038574,1400.0,华贵大麦甜蜜家定期寿险,20年,369.00,众惠全能百万医疗2019款,个人版
10,20191227210038506,4845.0,百年康惠保(2020版)重大疾病保险,30年,4845.00,百年康惠保(2020版)重大疾病保险,30年


## 寿险数据

In [37]:
order_life_20 = pd.read_excel(dir_order_life + 'order_life_20200514.xlsx',header=1)
order_life_20.rename(columns = {'行政录入日期':'date_input' ,'投保人':'tb_name' ,'被保险人':'bb_name','退保日期':'date_tb'
                              ,'现代理人编码':'ucode','标准保费':'bf','代理人所在机构':'area_type','险种名称':'pname'},inplace=True)
order_life_20['mon'] = order_life_20.date_input.map(date_bus)

In [43]:
cust_all.head()

,PROPOSALCONTNO,PERIODFLAG,INSUREPERIOD,INSURE_SEX,INSURE_AGE,INSURED_SEX,INSURED_AGE
0,202031,1年,1年,女,25,女,25
1,310A1214328,1年,1年,女,44,女,44
2,672001000247728,1年,200年,女,56,女,56
3,672001000448896,1年,200年,女,36,女,7
4,600001267509001,5年,至200岁,女,30,女,30


In [45]:
order_life_20.loc[order_life_20.投保单号 =='202031']

,序号,保单状态,date_tb,回单日期,保单录入日期,承保日期,预收日期,date_input,销售团队代码,销售团队名称,...,机构,职级,入司日期.1,入司月,营业部代码.1,营业部名称.1,是否营业中心.1,师傅代码,师傅姓名,mon


In [46]:
order_life_20.loc[order_life_20.保单号 =='202031']

,序号,保单状态,date_tb,回单日期,保单录入日期,承保日期,预收日期,date_input,销售团队代码,销售团队名称,...,机构,职级,入司日期.1,入司月,营业部代码.1,营业部名称.1,是否营业中心.1,师傅代码,师傅姓名,mon


In [42]:
order_life_20[['投保单号','保单号','date_input','pname']]

,投保单号,保单号,date_input,pname
0,M110596589602488,110008437888,2020-05-10,光大永明童佳保尊享版重大疾病保险
1,662003000059078,500006985008,2020-03-26,信泰如意享六福版养老年金保险
2,1187011444234731,P000000080083204,2020-01-05,富德生命倍享健康重大疾病保险基本责任
3,662003000059092,500006990392,2020-03-27,信泰完美人生守护尊享版重大疾病保险
4,1033010000863584,8808120819600688,2020-01-17,华夏华夏福多倍2.0版重大疾病保险
5,00046629025203036,NaN,2020-05-07,天安人寿健康源2019增强版终身重大疾病保险
6,1033010000913455,8808119071011088,2020-03-20,华夏医保通普惠版医疗保险
7,M270512341303788,270001371588,2020-02-10,光大永明百万安康2019版医疗保险
8,600002219121001,648605000021906478,2020-03-28,附加恒久安心住院医疗保险
9,600001625584001,648604000015455128,2019-12-28,附加恒久安心住院医疗保险


## 峰会新单预警

In [57]:
user_fh = order_20.loc[order_20.is_renew =='新单',:].groupby(['uid']).bf.sum().reset_index().sort_values('bf',ascending=False)

In [59]:
user_all.rename(columns={'用户ID':'uid'},inplace=True)

In [64]:
res = pd.merge(user_fh,user_all,how='left',on='uid')
res.loc[res.工号.notna(),:].to_excel('12.xlsx')

In [60]:
user_all

,ID,uid,工号,姓名,性别,年龄,LOCATIONNAME,MGMTDIVISIONNAME
0,1,c1bfc8aadf1047bfa38270c09f533e08,640284553.0,季成龙,男,32.0,大童上海区域中心,上海市区营业部
1,2,c33d709448a5446e9fb3bc1e6264e8ca,640284555.0,何春坪,男,38.0,大童四川区域中心,四川本部营业部
2,3,d394783e39ba443fb636a46bf2b89a89,640284556.0,窦剑武,男,38.0,大童辽宁区域中心,大童辽宁红兴车世界营业部
3,4,9b052accef394087a66140f6908101af,640284563.0,杨金玲,女,40.0,大童辽宁区域中心,辽宁本部营业部
4,5,e6e836c5abb7468bad6132faeefa4203,640284565.0,张慧霞,女,30.0,大童青岛区域中心,青岛西海岸营业部
5,6,2754e7176d354573a91bb253bd42fdc5,640284566.0,沈杰,女,36.0,大童北京区域中心,北京朝阳营业部
6,7,28de87a5da6d4b21be0423119ef8a768,640284567.0,关银强,男,38.0,大童大连区域中心,大连市区营业部
7,8,6758c3ecda7049839f2bd4480249a293,640290846.0,张红霞,女,34.0,大童北京区域中心,北京市区营业部
8,9,f124a25fc7a24c51957b66df7f27e520,640290849.0,吴咏梅,女,53.0,大童云南区域中心,云南E营业部
9,10,70e74bd9346d4479aeab15ea2f8972a3,640286392.0,曾祥俊,女,50.0,大童四川区域中心,四川本部营业部


## 快保用户行为价值贡献表

### 数据读取

In [122]:
col = ['order','bf','ucode','time_pay','tb_name','bb_name','is_insider','channel_src','tb_id','bb_id','date_pay','uid']
order_all = pd.concat([order_2018[col],order_2019[col],order_sum[col]])
order_all['mon'] = order_all.date_pay.map(date_bus)

In [46]:
## 寿险用户和寿险保单数据
user_offline_all = pd.read_excel(dir_user + 'user_info_offline_20200426.xlsx',header=2)

In [47]:
user_offline = user_offline_all.copy()
user_offline.rename(columns = {'代理人编码':'ucode','报聘日期':'date_join'},inplace=True)
user_offline['mon_join'] = user_offline.date_join.map(date_bus)

In [ ]:
order_detail = pd.read_excel(dir_order_life +'order_life_20200428.xlsx',header=1)

In [151]:
order_detail.rename(columns = {'代理人代码':'ucode','代理人所在机构':'area_type','代理人姓名':'uname','标准保费':'bf_std','行政录入日期':'date_pay',
                               '险种名称':'pname','投保人':'tb_name','被保险人':'bb_name'},inplace=True)
order_detail['mon'] = order_detail.date_pay.map(date_bus)

In [65]:
user_online = pd.read_csv(dir_user + 'user_info_online_20200429.csv',low_memory=False,parse_dates=['入职时间'])

In [89]:
user_online.loc[user_online.认证时间=='--','认证时间'] =user_online.loc[user_online.认证时间=='--','注册时间']
user_online['date_attest'] = pd.to_datetime(pd.to_datetime(user_online.认证时间).dt.strftime('%Y-%m-%d'))
user_online.rename(columns = {'用户ID':'uid','工号':'ucode','入职时间':'date_join'},inplace=True)
user_online['mon_attest'] = user_online.date_attest.map(date_bus)
user_online['days_delta'] = (user_online.date_join - user_online.date_attest).map(lambda x:x.days)
user_online.ucode.fillna(0,inplace=True)
user_online['ucode'] = user_online.ucode.map(lambda x:str(int(x)).strip())

### 业务情况

In [84]:
## 先认证快保后入司人力
user_online.loc[(user_online.days_delta >=2) 
                & (user_online.mon_attest >= '2018-08'),:].groupby('mon_attest').uid.nunique().reset_index().tail()


,mon_attest,uid
17,2020-01,68
18,2020-02,74
19,2020-03,342
20,2020-04,812
21,2020-05,20


In [79]:
## 核心开单人力
order_detail.groupby(['mon']).ucode.nunique().reset_index()

,mon,ucode
0,2020-01,6472
1,2020-02,3453
2,2020-03,5551
3,2020-04,5784
4,2020-05,911


In [91]:
## 贡献情况-线上保费
uid_att_join = user_online.loc[(user_online.days_delta >=2) 
                & (user_online.mon_attest >= '2018-08'),:].uid.unique()
ucode_att_join = user_online.loc[(user_online.days_delta >=2) 
                & (user_online.mon_attest >= '2018-08'),:].ucode.unique()
order_sum.loc[order_sum.uid.isin(uid_att_join),:].groupby('mon').bf.sum().tail().reset_index()

,mon,bf
0,2020-01,2013658.89
1,2020-02,2084683.40
2,2020-03,1943744.38
3,2020-04,2812820.57


In [96]:
## 贡献情况-线下人力保费
order_detail.loc[order_detail.ucode.isin(ucode_att_join),:].groupby(['mon']).agg({'ucode':'nunique','bf_std':'sum'}).reset_index()

,mon,ucode,bf_std
0,2020-01,449,7660641.70
1,2020-02,235,3272164.24
2,2020-03,417,5654310.93
3,2020-04,655,9427975.51
4,2020-05,103,935052.17


### 活跃情况

In [103]:
login = pd.read_excel(dir_login + 'login_2020_0326_0425.xlsx')
login['mon'] =pd.to_datetime(login.date_login).map(date_bus)
login.groupby(['mon','is_insider']).uid.nunique().reset_index().tail()

,mon,is_insider,uid
0,2020-04,0,41364
1,2020-04,1,34252


In [101]:
## 司内外日军活跃人数
res = login.groupby(['mon','is_insider','date_login']).agg({'uid':'nunique'}).reset_index().groupby(['mon','is_insider']).agg({'date_login':'count','uid':'sum'}).reset_index()
res['days_avg_act'] =res.uid / res.date_login
res.pivot(index='mon',columns = 'is_insider',values = 'days_avg_act')

is_insider,0,1
mon,,
2020-03,6426.37931,17154.068966


In [105]:
## 司内外7天 或者 20天以上活跃人数
res = login.groupby(['mon','uid','is_insider']).agg({'date_login':'nunique'}).reset_index()
res.loc[res.date_login>=20,:].groupby(['mon','is_insider']).size().reset_index().pivot(index='mon',columns='is_insider',values=0).tail()

is_insider,0,1
mon,,
2020-04,2216,13703


### 签到情况

In [108]:
## 1.司内外签到用户数
score = pd.read_csv(dir_score + 'score_20200326_20200425.csv',parse_dates=['完成时间'])

In [112]:
score['date_complete'] = pd.to_datetime(score.完成时间.dt.strftime('%Y-%m-%d'))
score['mon'] = score.date_complete.map(date_bus)
score.loc[score.任务名称 =='每日签到',:].groupby(['mon','是否司内']).agg({'快保ID':'nunique'}).tail(24).reset_index().pivot(index='mon',columns = '是否司内',values='快保ID')

是否司内,否,是
mon,,
2020-04,6624,19252


In [115]:
## 2.日均签到用户数
res = score.loc[(score.任务名称 =='每日签到'),:].groupby(['mon','是否司内','date_complete']).agg({'快保ID':'nunique'}).reset_index()
res = res.groupby(['mon','是否司内']).agg({'date_complete':'nunique','快保ID':'sum'}).reset_index()
res['avg_qd'] = res.快保ID/res.date_complete
res

,mon,是否司内,date_complete,快保ID,avg_qd
0,2020-04,否,31,67831,2188.096774
1,2020-04,是,31,318873,10286.225806


In [118]:
## 签到20天的人数
res = score.loc[(score.任务名称 =='每日签到'),:].groupby(['mon','快保ID','是否司内']).agg({'date_complete':'nunique'}).reset_index()
res.loc[res.date_complete >= 20,:].groupby(['mon','是否司内']).agg({'快保ID':'nunique'}).reset_index().pivot(index='是否司内',columns = 'mon',values='快保ID')

mon,2020-04
是否司内,
否,1939
是,9254


### 客户情况

In [123]:
order_all['rns'] =  order_all['date_pay'].groupby(order_all['tb_id']).rank(ascending=True,method='first')
order_online_01 = order_all.loc[order_all.rns ==1,['tb_id','date_pay','is_insider']].copy()
order_all['rns'] =  order_all['date_pay'].groupby(order_all['bb_id']).rank(ascending=True,method='first')
order_online_02 = order_all.loc[order_all.rns ==1,['bb_id','date_pay','is_insider']].copy()

order_online_01.columns = ['cust_id','date_pay_first','is_insider']
order_online_02.columns = ['cust_id','date_pay_first','is_insider']
order_online = pd.concat([order_online_01,order_online_02]).drop_duplicates()

order_online['rns'] = order_online['date_pay_first'].groupby(order_online['cust_id']).rank(ascending=True,method='first')

cust_info_online = order_online.loc[order_online.rns ==1,['cust_id','date_pay_first','is_insider']]
cust_info_online['mon'] = cust_info_online.date_pay_first.map(date_bus)
cust_info_online.fillna('2016-05',inplace=True)

In [124]:
##每月新客数量
res_newcust_sum = cust_info_online.groupby(['mon']).size().reset_index().tail(12)
res_newcust_insider = cust_info_online.groupby(['mon','is_insider']).size().reset_index().tail(24).pivot(index='mon',columns = 'is_insider',values=0).reset_index()[['mon','是','否']]
pd.merge(res_newcust_sum,res_newcust_insider,how='left',on='mon').tail()

,mon,0,是,否
7,2019-12,82297,50295,32002
8,2020-01,69044,45782,23262
9,2020-02,60309,51133,9176
10,2020-03,98103,82651,15452
11,2020-04,58999,36399,22600


In [125]:
## 每月总客户数量
cust_tb = order_all.groupby(['date_pay','tb_id']).size().reset_index()
cust_bb = order_all.groupby(['date_pay','bb_id']).size().reset_index()
cust_tb.columns = ['date_pay','cust_id','num']
cust_bb.columns = ['date_pay','cust_id','num']

col = ['date_pay','cust_id']
cust_num_sum = pd.concat([cust_tb[col],cust_bb[col]]).drop_duplicates()
cust_num_sum['mon'] = cust_num_sum.date_pay.map(date_bus)
cust_num_sum.mon.fillna('2016-05',inplace=True)
cust_num_sum = cust_num_sum[['mon','cust_id']].drop_duplicates()

cust_num_sum.groupby('mon').size().reset_index().tail()

,mon,0
23,2019-12,136356
24,2020-01,126487
25,2020-02,122135
26,2020-03,180835
27,2020-04,121861


In [126]:
##每月老客数量
res1 = pd.merge(cust_num_sum,cust_info_online[['cust_id','date_pay_first','mon']],how='left',on=['mon','cust_id'])
res1.loc[res1.date_pay_first.isna(),'cust_type'] ='老客'
res1.cust_type.fillna('新客',inplace=True)
res2 = pd.merge(res1,cust_info_online[['cust_id','is_insider']],how='left',on='cust_id')

res_ocust_sum = res2.loc[res2.cust_type == '老客',:].groupby(['mon']).size().reset_index().tail(12)
res_ocust_insider = res2.loc[res2.cust_type == '老客',:].groupby(['mon','is_insider']).size().reset_index().tail(24).pivot(index='mon',columns = 'is_insider',values=0).reset_index()[['mon','是','否']]
pd.merge(res_ocust_sum,res_ocust_insider,how='left',on='mon')

,mon,0,是,否
0,2019-05,60320,39051,21269
1,2019-06,64010,43406,20604
2,2019-07,60103,41412,18691
3,2019-08,68787,48419,20368
4,2019-09,94179,64546,29633
5,2019-10,60041,41703,18338
6,2019-11,59332,41604,17728
7,2019-12,54059,38590,15469
8,2020-01,57443,41584,15859
9,2020-02,61826,49529,12297


In [127]:
## 新老客保费
order_all['mon'] = order_all.date_pay.map(date_bus)
col = ['order','date_pay','mon','bf','tb_id','bb_id','is_insider']
res_01 = pd.merge(order_all[col],cust_info_online,how='left',left_on=['mon','tb_id'],right_on=['mon','cust_id'])
res_02 = pd.merge(res_01,cust_info_online,how='left',left_on=['mon','bb_id'],right_on=['mon','cust_id'],suffixes=['_tb','_bb'])

res_02.loc[(res_02.cust_id_tb.notna()) & (res_02.cust_id_bb.notna()),'is_new_cust'] = '新客'
res_02.is_new_cust.fillna('老客',inplace=True)

res_ncust_insider = res_02.loc[res_02.is_new_cust =='新客',:].groupby(['mon','is_new_cust','is_insider_x']).bf.sum().reset_index().tail(24).pivot(index='mon',columns='is_insider_x',values='bf').reset_index()
res_ocust_insider = res_02.loc[res_02.is_new_cust =='老客',:].groupby(['mon','is_new_cust','is_insider_x']).bf.sum().reset_index().tail(24).pivot(index='mon',columns='is_insider_x',values='bf').reset_index()
res_sum = pd.merge(res_ncust_insider,res_ocust_insider,how='left',on='mon')

res_bf = res_02.groupby(['mon','is_new_cust']).bf.sum().reset_index().tail(24).pivot(index='mon',columns = 'is_new_cust',values='bf').reset_index()
pd.merge(res_bf,res_sum,how='left',on='mon')[['mon','新客','是_x','否_x','老客','是_y','否_y']]

,mon,新客,是_x,否_x,老客,是_y,否_y
0,2019-05,2.017580e+07,1.376635e+07,6409452.82,1.362636e+07,1.051595e+07,3110411.39
1,2019-06,2.047645e+07,1.501493e+07,5461514.54,1.543290e+07,1.211227e+07,3320628.96
2,2019-07,1.776492e+07,1.303341e+07,4731507.59,1.413387e+07,1.102361e+07,3110267.99
3,2019-08,1.776244e+07,1.336868e+07,4393762.68,1.752786e+07,1.347398e+07,4053878.41
4,2019-09,2.106582e+07,1.553778e+07,5528043.46,1.950285e+07,1.442061e+07,5082240.74
5,2019-10,1.734281e+07,1.336235e+07,3980454.84,1.618709e+07,1.253543e+07,3651650.97
6,2019-11,1.852868e+07,1.442770e+07,4100975.22,1.945685e+07,1.524612e+07,4210735.11
7,2019-12,1.888982e+07,1.480447e+07,4085351.72,1.893903e+07,1.499284e+07,3946183.63
8,2020-01,1.789971e+07,1.419137e+07,3708342.23,2.002434e+07,1.556410e+07,4460235.32
9,2020-02,1.648124e+07,1.404621e+07,2435029.02,2.220023e+07,1.824564e+07,3954587.20


In [130]:
## 支付方式
res = order_sum.groupby(['mon','channel_src','is_insider']).agg({'order':'count','bf':'sum'}).reset_index()
res['key'] =res.channel_src + '_' + res.is_insider
res_cnt = res.pivot(index='mon',columns = 'key',values= 'order')

res_cnt['APP'] = res_cnt.APP_否 + res_cnt.APP_是
res_cnt['网页'] = res_cnt.网页_否 + res_cnt.网页_是
res_cnt['批量'] = res_cnt.批量_否 + res_cnt.批量_是

col = ['APP','APP_是','APP_否','网页','网页_是','网页_否','批量','批量_是','批量_否']
res_cnt[col].tail(12)

key,APP,APP_是,APP_否,网页,网页_是,网页_否,批量,批量_是,批量_否
mon,,,,,,,,,
2020-01,77054,52052,25002,25541,17970,7571,11744,8919,2825
2020-02,84811,70766,14045,28284,24161,4123,577,425,152
2020-03,76420,52551,23869,95743,89056,6687,302,247,55
2020-04,88562,56517,32045,20578,13511,7067,3576,3158,418


### 客户贡献情况

In [174]:
order_all['rns'] = order_all['date_pay'].groupby(order_all['uid']).rank(ascending=False,method='first')
user_info = order_all.loc[order_all.rns ==1,['uid','ucode']].copy()

In [203]:
order_sum['uid'] = order_sum.uid.map(lambda x:x.strip())
user_info['uid'] = user_info.uid.map(lambda x:x.strip())
order_sum.ucode.fillna(0,inplace=True)
order_sum['ucode'] = order_sum.ucode.map(lambda x:str(int(x)).strip())

order_on = pd.merge(order_sum,user_info,how='left',on='uid')
order_on.ucode_y.fillna(0,inplace=True)
order_on['ucode'] = order_on.ucode_y.map(lambda x:str(int(x)).strip())
order_off = order_detail.copy()
order_off['ucode']
order_off['ucode'] = order_off.ucode.map(lambda x:str(int(x)).strip())

In [206]:
order_on['cust_id_tb'] = order_on.ucode + '_'+order_on.tb_name
order_on['cust_id_bb'] = order_on.ucode + '_'+order_on.bb_name

order_off['cust_id_tb'] = order_off.ucode + '_'+order_off.tb_name
order_off['cust_id_bb'] = order_off.ucode + '_'+order_off.bb_name

In [245]:
## 线上线下同时开单的代理人数
res1 = order_sum.groupby(['mon','ucode']).bf.sum().reset_index()
res2 = order_off.groupby(['mon','ucode']).bf_std.sum().reset_index()
res = pd.merge(res1,res2,how='left',on=['mon','ucode'])
res.loc[res.bf_std.notna(),:].groupby(['mon']).ucode.nunique().reset_index()

,mon,ucode
0,2020-01,4678
1,2020-02,2870
2,2020-03,4494
3,2020-04,4341


In [244]:
## 线上线下同时开单的客户数
res1 = order_on.groupby(['date_pay','cust_id_bb']).bf.sum().reset_index()
res1['mon'] = res1.date_pay.map(date_bus)
res2 = order_off.groupby(['date_pay','cust_id_bb']).bf_std.sum().reset_index()
res2['mon'] = res2.date_pay.map(date_bus)
res = pd.merge(res1,res2,how='left',on=['mon','cust_id_bb'])
res.loc[res.bf_std.notna(),:].groupby(['mon']).cust_id_bb.nunique().reset_index()

,mon,cust_id_bb
0,2020-01,3236
1,2020-02,2095
2,2020-03,3314
3,2020-04,3085


In [218]:
##线上客户首单时间
order_on['rns'] = order_on['date_pay'].groupby(order_on['cust_id_tb']).rank(ascending=True,method='first')
cust_id_tb = order_on.loc[order_on.rns ==1,['cust_id_tb','date_pay','area_type','ucode','uid']].copy()
cust_id_tb.columns = ['cust_id_bb','date_pay','area_type','ucode','uid']
order_on['rns'] = order_on['date_pay'].groupby(order_on['cust_id_bb']).rank(ascending=True,method='first')
cust_id_bb = order_on.loc[order_on.rns ==1,['cust_id_bb','date_pay','area_type','ucode','uid']].copy()
cust_id_on = pd.concat([cust_id_tb,cust_id_bb])

In [219]:
##线下客户首单时间
order_off['rns'] = order_off['date_pay'].groupby(order_off['cust_id_tb']).rank(ascending=True,method='first')
cust_id_tb = order_off.loc[order_off.rns ==1,['cust_id_tb','date_pay','area_type','ucode']].copy()
cust_id_tb.columns = ['cust_id_bb','date_pay','area_type','ucode']
order_off['rns'] = order_off['date_pay'].groupby(order_off['cust_id_bb']).rank(ascending=True,method='first')
cust_id_bb = order_off.loc[order_off.rns ==1,['cust_id_bb','date_pay','area_type','ucode']].copy()
cust_id_off = pd.concat([cust_id_tb,cust_id_bb])

In [222]:
cust_id_on['rns'] = cust_id_on['date_pay'].groupby(cust_id_on['cust_id_bb']).rank(ascending=True,method='first')
cust_id_off['rns'] = cust_id_off['date_pay'].groupby(cust_id_off['cust_id_bb']).rank(ascending=True,method='first')

In [229]:
cust_id_on = cust_id_on.loc[cust_id_on.rns ==1,:].copy()
cust_id_off = cust_id_off.loc[cust_id_off.rns ==1,:].copy()
cust_res = pd.merge(cust_id_on,cust_id_off[['cust_id_bb','date_pay']],how='left',on='cust_id_bb')

In [231]:
cust_res['days'] = (cust_res.date_pay_y - cust_res.date_pay_x).map(lambda x:x.days)

In [248]:
## 线上客户首单时间  与 线下客户首单时间
cust_res['mon'] = cust_res.date_pay_x.map(date_bus)
cust_res.loc[cust_res.days >=2,:].groupby('mon').cust_id_bb.nunique().reset_index()

,mon,cust_id_bb
0,2020-01,2160
1,2020-02,2702
2,2020-03,2238
3,2020-04,555


In [251]:
order_off.loc[order_off.cust_id_bb.isin(cust_res.loc[cust_res.days >=2,:].cust_id_bb.unique()),:].groupby('mon').bf_std.sum().reset_index()

,mon,bf_std
0,2020-01,3077202.40
1,2020-02,4980967.93
2,2020-03,14209179.62
3,2020-04,13785197.52
4,2020-05,1602306.27


## 活动成本计算

### 福利券

In [125]:
order_sum = order_2019_need.copy()

In [14]:
order_sum['ticket_name_bak'] = order_sum.ticket_name 
order_sum.loc[order_sum.ticket_name_bak.notna(),'ticket_name_bak'] = order_sum.loc[order_sum.ticket_name_bak.notna(),'ticket_name_bak'].map(lambda x:x.strip())

In [17]:
order_sum.ticket_rate.fillna('无',inplace=True)
order_sum.ticket_value.fillna('无',inplace=True)
order_sum.loc[(order_sum.date_pay >= pd.to_datetime('2020-03-26'))
                       & (order_sum.date_pay <= pd.to_datetime('2020-04-25'))
                       ,:].groupby(['ticket_name','ticket_rate','ticket_value']).agg({'uid':'nunique','order':'count','tgf_X':'sum','bf':'sum'}).reset_index().to_excel('12.xlsx',encoding='gbk')

In [16]:
order_need.ticket_rate.fillna('无',inplace=True)
order_need.ticket_value.fillna('无',inplace=True)
order_need.loc[(order_need.date_pay >= pd.to_datetime('2019-10-26'))
                    & (order_need.date_pay <= pd.to_datetime('2019-11-25'))
                       ,:].groupby(['ticket_name','ticket_rate','ticket_value']).agg({'uid':'nunique','order':'count','tgf_X':'sum','bf':'sum'}).reset_index().to_excel('11.xlsx',encoding='gbk')

NameError: name 'order_need' is not defined

### 推广费成本

In [96]:
agg_fun = {'bf':'sum','tgf_A':'sum','tjf':'sum','tgf_B':'sum','tgf_b1':'sum','tgf_b2':'sum','tgf_B_cal':'sum','tgf_X':'sum'}
order_sum['tgf_B_cal'] = order_sum.tgf_b1 + order_sum.tgf_b2
order_sum.loc[order_sum.uid !='U00000000',:].groupby('mon').agg(agg_fun).reset_index().tail()

,mon,bf,tgf_A,tjf,tgf_B,tgf_b1,tgf_b2,tgf_B_cal,tgf_X
0,2020-01,3.781093e+07,8.259251e+06,411120.00,4.271207e+06,3.462658e+06,733720.97,4.196379e+06,769427.74
1,2020-02,3.847426e+07,6.919081e+06,344222.17,3.445791e+06,3.154371e+06,291419.72,3.445791e+06,1062971.41
2,2020-03,3.821151e+07,7.849676e+06,390968.71,3.719637e+06,3.310317e+06,409319.24,3.719637e+06,999226.52
3,2020-04,4.329097e+07,9.150784e+06,455351.82,4.557177e+06,3.946183e+06,610993.83,4.557177e+06,727729.06


In [97]:
order_all = pd.merge(order_sum,ptype,how='left',on='pname')

In [101]:
agg_fun = {'bf':'sum','tgf_A':'sum','tjf':'sum','tgf_B':'sum','tgf_b1':'sum','tgf_b2':'sum','tgf_X':'sum'}
order_all.loc[order_all.mon =='2020-04',:].groupby(['plabel_term']).agg(agg_fun)

,bf,tgf_A,tjf,tgf_B,tgf_b1,tgf_b2,tgf_X
plabel_term,,,,,,,
0,34216603.66,6.599647e+06,327647.63,2144577.63,1623044.96,521532.67,479086.10
1,9194814.22,2.575414e+06,128807.68,2423067.51,2332891.57,90175.71,248642.96


In [89]:
order_all.loc[order_all.plabel_term.isna(),:].groupby('pname').size().reset_index()

,pname,0


### 指定产品的推广费成本

In [44]:
pname_list = ['众惠全能百万医疗2019款','众安尊享e生2019版','平安个人意外保障计划','复星联合康乐一生(C 款升级款)',
              '百年康惠保(旗舰版)','富德生命童宝保少儿重大疾病保险','华贵大麦定期寿险']
agg_fun = {'bf':'sum','tgf_A':'sum','tjf':'sum','tgf_B':'sum','tgf_b1':'sum','tgf_b2':'sum'}
col = ['pname','bf','tgf_A','tgf_B','tgf_rate_B','plan','date_pay']
res = order_sum.loc[(order_sum.date_pay >=pd.to_datetime('2019-07-09'))
              & (order_sum.date_pay <= pd.to_datetime('2019-07-22'))
              & (order_sum.plan.isin(['20年','30年']))
              & (order_sum.pname.str.contains('妈咪')),col].copy()
res['tgf_rate_b_cal'] = res.tgf_B/res.bf
res['b2'] = res.bf * 0.1
res.b2.sum()
# .groupby(['pname']).agg(agg_fun).reset_index().sort_values('bf',ascending=False).to_excel('12.xlsx',encoding='gbk',index=0)

34335.14

### 产品利润

In [316]:
pname_profit = order_sum.loc[order_sum.mon =='2019-08',:].groupby(['pname','plan','tgf_rate_A']).agg({'bf':'sum','tgf_A':'sum','tgf_B':'sum'}).reset_index()

In [317]:
pname_profit['rate_A'] = pname_profit.tgf_A/pname_profit.bf
pname_profit['income_rate'] = pname_profit.rate_A / 0.7
pname_profit['income']= pname_profit.bf * pname_profit.income_rate
pname_profit['profit'] = pname_profit.income/1.06-(pname_profit.tgf_A + pname_profit.tgf_B)
pname_profit['profit_rate'] = pname_profit.profit / pname_profit.bf

In [319]:
pname_profit.sort_values('profit_rate',ascending=False).tail(20)

,pname,plan,tgf_rate_A,bf,tgf_A,tgf_B,rate_A,income_rate,income,profit,profit_rate
409,百年康惠保（尊享版）重大疾病保险,10年,20%,21475.00,4295.00,2147.50,0.200000,0.285714,6135.714286,-654.090296,-0.030458
140,复星联合妈咪保贝少儿重疾保险,10年,20%,36573.10,7314.62,3657.31,0.200000,0.285714,10449.457143,-1113.951563,-0.030458
63,众安尊享e生2019版,家庭版,20%,389396.00,77879.20,40828.80,0.200000,0.285714,111256.000000,-13749.509434,-0.035310
149,复星联合康乐一生（2019版）重大疾病保险,15年,30%,15105.00,4531.50,2265.74,0.300000,0.428571,6473.571429,-690.097143,-0.045687
410,百年康惠保（尊享版）重大疾病保险,20年,35%,80142.00,28049.70,14425.56,0.350000,0.500000,40071.000000,-4672.429811,-0.058302
411,百年康惠保（尊享版）重大疾病保险,30年,35%,80570.00,28199.50,14502.60,0.350000,0.500000,40285.000000,-4697.383019,-0.058302
406,百年康惠保(旗舰版),20年,30%,473936.00,142180.80,77817.52,0.300000,0.428571,203115.428571,-28379.991159,-0.059881
275,富德生命爱相伴费用补偿医疗保险(2019 版),1年,10%,81513.00,8151.30,8151.30,0.100000,0.142857,11644.714286,-5317.020485,-0.065229
408,百年康惠保(旗舰版),30年,30%,677984.00,203395.20,116300.60,0.300000,0.428571,290564.571429,-45578.279784,-0.067226
145,复星联合妈咪保贝少儿重疾保险,30年,45%,335597.90,151018.85,76213.18,0.449999,0.642856,215741.214286,-23702.582561,-0.070628


In [313]:
order_sum.loc[order_sum.pname.str.contains('基因')].groupby(['pname','bf']).size().reset_index()

,pname,bf,0
0,众安生命儿童兴趣基因检测,258.0,29
1,众安生命儿童兴趣基因检测,488.0,241
2,众安生命儿童兴趣基因检测（团购）,206.4,64
3,众安生命儿童兴趣基因检测（团购）,232.2,14
4,众安生命儿童兴趣基因检测（团购）,390.4,1621
5,众安生命儿童兴趣基因检测（团购）,439.2,69
6,众安生命儿童安全用药基因检测,390.4,3
7,众安生命儿童安全用药基因检测,488.0,4
8,众安生命百项全能基因组检测,488.0,36
9,众安生命百项全能基因组检测（团购）,429.4,938


In [309]:
pname_profit.head()

,pname,tgf_rate_A,bf,tgf_A,tgf_B,rate_A,income_rate,income,profit
0,“安盛户外”境内旅行保险,28%,2554.0,715.12,0.0,0.28,0.400000,1021.600000,248.653585
1,“童”享阳光家庭财产保险A款,38%,2700.0,1026.00,0.0,0.38,0.542857,1465.714286,356.749326
2,“童”享阳光家庭财产保险B款,38%,9212.0,3500.56,0.0,0.38,0.542857,5000.800000,1217.175849
3,“阳光家园”家庭财产火灾保险,21%,3420.0,718.20,0.0,0.21,0.300000,1026.000000,249.724528
4,中信保诚「亲恩保」恶性肿瘤疾病保险,20%,13272.5,2654.50,0.0,0.20,0.285714,3792.142857,922.993261


### 积分成本

In [5]:
score_01 = pd.read_excel(dir_score + 'score_20191226_20200125.xlsx')
score_01.pivot_table(index=['module_name','job_name'],columns = ['sinei'],values='point_num',aggfunc = 'sum').reset_index().to_excel('12.xlsx')

In [6]:
score_04 = pd.read_csv(dir_score + 'score_20200326_20200425.csv')

In [10]:
score_04.groupby(['模块名称','任务名称']).积分.sum().reset_index().head

,模块名称,任务名称,积分
0,任务中心,任务奖励,5010426
1,任务中心,宝箱奖励,2467102
2,任务中心,每日签到,2903895
3,任务中心,补签,-1025600
4,兑换道具,兑换模板,-178000
5,出单送保贝产品二,出单送保贝,8160000
6,出单送宝贝产品一,出单送保贝,2088000
7,商品兑换,100元福利券,-620000
8,商品兑换,10元京东e卡,-78020
9,商品兑换,20元京东e卡,-6640


In [12]:
score_04.loc[score_04.任务名称.str.contains('春花'),:].groupby('积分').size().reset_index()

,积分,0
0,-7500,12
1,-5200,35
2,-4982,1
3,-3453,1
4,-3043,1
5,-3028,1
6,-2955,1
7,-2943,1
8,-2709,1
9,-2462,1


In [25]:
score_04.loc[score_04.模块名称 !='用户打赏'].pivot_table(index=['模块名称','任务名称'],columns =['是否司内'],values='积分',aggfunc='sum').reset_index().to_excel('12.xlsx')

In [107]:
score['完成时间'] = pd.to_datetime(pd.to_datetime(score.完成时间).dt.strftime('%Y-%m-%d'))
score['mon'] = score.完成时间.map(date_bus)

In [109]:
score.模块名称.unique()

array(['任务中心', '成长营奖励', '商品兑换', '积分发放', '用户打赏', '出单送宝贝产品一', '出单送保贝产品二',
       '兑换道具', '学习计划', '商品退款'], dtype=object)

In [111]:
score.groupby(['模块名称','任务名称','是否司内']).agg({'类型':'count','积分':'sum'}).reset_index().to_excel('12.xlsx',encoding='gbk',index=0)

In [105]:
score.loc[score.job_name.str.contains('京东'),:].groupby(['job_name','point_num']).agg({'jfly':'count'}).reset_index()

,job_name,point_num,jfly
0,10元京东e卡,-1660,9
1,20元京东e卡,-3320,1
2,20元京东e卡,-2000,30
3,30元京东e卡,-4980,10
4,30元京东e卡,-3000,30


In [144]:
score.loc[(score.mon >= '2019-01') & (score.job_name =='每日签到'),:].groupby(['location','mon']).user_id.nunique().reset_index().pivot(index='location',columns = 'mon',values='user_id').to_excel('12.xlsx',encoding='gbk')

In [82]:
score.groupby(['module_name','job_name']).agg({'user_id':'count','point_num':'sum'}).reset_index().to_excel('12.xlsx')

## 平台基础保费测算

### 调整非加点商品

In [21]:
## 2月份调整商品
order_sum.loc[(order_sum.mon =='2020-02') 
              & (order_sum.pname == '复星联合爱无忧意外伤害保险'),'tgf_b2'] = 0.1
## 3月份调整商品
order_sum.loc[(order_sum.date_pay >= pd.to_datetime('2020-03-11'))
              & (order_sum.date_pay <= '2020-03-25') 
              & (order_sum.pname =='复星联合妈咪保贝少儿重大疾病保险'),'tgf_b2'] = 0.1
order_sum.loc[(order_sum.mon =='2020-03') 
              & (order_sum.pname =='众安尊享e生2020版'),'tgf_b2'] = 0.1
## 4月份调整商品
order_sum.loc[(order_sum.date_pay >= pd.to_datetime('2020-03-26'))
              & (order_sum.date_pay <= '2020-04-13') 
              & (order_sum.pname =='百年康惠保(2020版)重大疾病保险'),'tgf_b2'] = 0.1
order_sum.loc[(order_sum.date_pay >= pd.to_datetime('2020-03-26'))
              & (order_sum.date_pay <= '2020-04-13') 
              & (order_sum.pname =='国富人寿嘉和保重大疾病保险'),'tgf_b2'] = 0.1
order_sum.loc[(order_sum.date_pay >= pd.to_datetime('2020-03-26'))
              & (order_sum.date_pay <= '2020-04-13') 
              & (order_sum.pname =='百年康惠保（青春定期版）'),'tgf_b2'] = 0.1
order_sum.loc[(order_sum.date_pay >= pd.to_datetime('2020-03-26'))
              & (order_sum.date_pay <= '2020-04-13') 
              & (order_sum.pname =='复星联合妈咪保贝少儿重大疾病保险'),'tgf_b2'] = 0.1
order_sum.loc[(order_sum.date_pay >= pd.to_datetime('2020-03-26'))
              & (order_sum.date_pay <= '2020-04-13') 
              & (order_sum.pname =='复星联合倍吉星重大疾病保险'),'tgf_b2'] = 0.1
order_sum.loc[(order_sum.date_pay >= pd.to_datetime('2020-03-26'))
              & (order_sum.date_pay <= '2020-04-13') 
              & (order_sum.pname =='百年超倍保重大疾病保险'),'tgf_b2'] = 0.1

In [22]:
## 区分活动订单
dic_act = order_sum.loc[(order_sum.tgf_b2 >0)
              & (order_sum.mon >='2020-03'),:].groupby(['date_pay','pname']).agg({'bf':'sum'}).reset_index()

## 福利券类型
ticket_type = pd.read_excel(dir_dic + '12_福利券类型.xlsx')

## 验证是否有福利券遗漏，忘记标类型
order_ticket = pd.merge(order_sum,ticket_type,how='left',on='ticket_name')
order_ticket.loc[(order_ticket.ticket_type.isna()),:].groupby('ticket_name').tgf_X.sum().reset_index()

,ticket_name,tgf_X
0,5月冲刺券,8970.0
1,E营业部专属大礼包,300.0
2,出单+10%,154.8


In [9]:
## 导出未标记的福利券
order_ticket.loc[(order_ticket.mon >='2020-02')
              & (order_ticket.ticket_type.isna()),:].groupby('ticket_name').tgf_X.sum().reset_index().to_excel('12.xlsx')

### 基础保费测算

In [23]:
order_detail = pd.merge(order_ticket,dic_act,how='left',on=['pname','date_pay'])

In [24]:
order_detail['order_type'] = np.nan
order_type_list = ['0_基础','1_商品活动','2_用户福利券']
order_detail.loc[order_detail.bf_y.notna(),'order_type'] = order_type_list[1]

order_detail.loc[(order_detail.order_type.isna())
                & (order_detail.ticket_type =='用户'),'order_type'] = order_type_list[2]

order_detail.order_type.fillna(order_type_list[0],inplace=True)
order_detail.groupby(['mon','order_type']).bf_x.sum().reset_index().pivot(index='mon',columns ='order_type',values='bf_x')

order_type,0_基础,1_商品活动,2_用户福利券
mon,,,
2020-01,2.560075e+07,NaN,12335215.83
2020-02,2.037843e+07,NaN,18314723.59
2020-03,1.924244e+07,7168912.6,11996628.47
2020-04,2.435758e+07,10103017.7,8964828.81
2020-05,1.618292e+07,4024161.0,12702607.51


In [25]:
order_detail.loc[order_detail.order_type==order_type_list[1],:].groupby(['mon','pname','date_pay']).bf_x.sum().reset_index().to_excel('12.xlsx')
dic_pname = order_detail.loc[order_detail.order_type==order_type_list[1],:].groupby(['mon','pname']).order.count().reset_index()
order_non_act = pd.merge(order_detail,dic_pname,how='left',on=['pname','mon'])
order_non_act.loc[order_non_act.order_y.notna(),:].groupby(['mon','pname','date_pay']).bf_x.sum().reset_index().to_excel('11.xlsx')

In [26]:
order_detail.ticket_type.fillna('未使用',inplace=True)
order_detail.groupby(['mon','order_type','ticket_type']).agg({'bf_x':'sum','tgf_X':'sum'}).reset_index().reset_index().tail(12)

,index,mon,order_type,ticket_type,bf_x,tgf_X
13,13,2020-04,0_基础,其他,1122826.98,116731.19
14,14,2020-04,0_基础,未使用,23234749.39,0.00
15,15,2020-04,1_商品活动,其他,394822.15,69377.20
16,16,2020-04,1_商品活动,未使用,6121002.11,0.00
17,17,2020-04,1_商品活动,用户,3587193.44,193266.61
18,18,2020-04,2_用户福利券,用户,8964828.81,349469.06
19,19,2020-05,0_基础,其他,1031895.98,111961.21
20,20,2020-05,0_基础,未使用,15151026.16,8994.80
21,21,2020-05,1_商品活动,其他,205944.80,50597.93
22,22,2020-05,1_商品活动,未使用,1909647.40,430.00


## 意外险用户品质

### 理赔数据

In [32]:
df1 = pd.read_excel(dir_sup+'意外险用户品质测算\\理赔订单列表 (1).xlsx')
df2 = pd.read_excel(dir_sup+'意外险用户品质测算\\理赔订单列表 (2).xlsx')
df3 = pd.read_excel(dir_sup+'意外险用户品质测算\\理赔订单列表 (3).xlsx')
order_pf = pd.concat([df1,df2,df3])
order_pf.rename(columns = {'订单号':'order','保单号':'order_origin'},inplace=True)

In [85]:
order_pf['order'] = order_pf.order.map(lambda x:str(x).strip())
order_pf['order_origin'] = order_pf.order_origin.map(lambda x:str(x).strip())

### 订单数据

In [33]:
col = ['order','order_origin','date_pay','bf','uid','pname','time_start','time_end']
order_2019 = order_2019.copy()
order_2020 = order_20.copy()
order_all = pd.concat([order_2019[col],order_2020[col]])
order_all['time_protect'] = (pd.to_datetime(order_all.time_end) - pd.to_datetime(order_all.time_start)).map(lambda x:x.days)
order_need = order_all.loc[(order_all.time_protect.isin([364,365])) 
                           & (order_all.order.isin(ct_order.order.unique())==False),:].copy()

### 用户基础

In [164]:
user_info = pd.read_csv(dir_user + 'user_info_online_20200429.csv',low_memory=False)
user_info.rename(columns = {'用户ID':'uid','工号':'ucode'},inplace=True)

user_info.loc[user_info.认证时间 =='--','认证时间']= user_info.loc[user_info.认证时间 =='--','注册时间']
user_info['long_attest'] = (pd.to_datetime('2020-05-20') -  pd.to_datetime(user_info.认证时间)).map(lambda x:x.days)
user_info = user_info[['uid','ucode','long_attest']].copy()
user_info.ucode.fillna(0,inplace=True)
user_info['ucode'] = user_info.ucode.map(lambda x:str(int(x)).strip())

In [165]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269180 entries, 0 to 269179
Data columns (total 3 columns):
uid            269180 non-null object
ucode          269180 non-null object
long_attest    269031 non-null float64
dtypes: float64(1), object(2)
memory usage: 6.2+ MB


### R13

In [192]:
user_r13 = pd.read_excel(dir_sup + '意外险用户品质测算\\R13_2020.04.30.xls')
user_r13 = user_r13[['代理人编码','继续率（13个月）']].copy()

In [193]:
user_r13.columns = ['ucode','r13']
user_r13.ucode.fillna(0,inplace=True)
user_r13['ucode'] = user_r13.ucode.map(lambda x:str(int(x)).strip())
user_r13 = user_r13.loc[user_r13.ucode != '0',:].copy()

### 职业类别

In [49]:
user_pro = pd.read_csv(dir_sup + 'user_profession.csv',encoding='utf-8',low_memory=False)
user_pro.订单号.replace('TW|T','',regex=True,inplace=True)
user_pro_need = user_pro[['订单号','被保人职业类别']].copy()
user_pro_need.columns = ['order','class_bb']

### 信息组合

In [127]:
order_pro = pd.merge(order_need,user_pro_need,how='left',on='order')
order_ptype = pd.merge(order_pro,ptype[['pname','plabel_need']],how='left',on='pname')
order_ptype.loc[order_ptype.plabel_need =='意外险_10+1','plabel_res'] = '0_意外险101'
order_ptype.plabel_res.fillna('1_其他产品',inplace=True)
order_pf_need = pd.merge(order_ptype,order_pf[['order_origin','order']],how='left',on='order_origin')

In [152]:
## 近半年出单人力
uid_order = order_pf_need.groupby(['uid','plabel_res']).order_x.count().reset_index().pivot('uid','plabel_res','order_x').reset_index()
uid_need_order = order_pf_need.loc[order_pf_need.date_pay >= pd.to_datetime('2019-11-20'),:].groupby(['uid','plabel_res']).order_x.count().reset_index().pivot('uid','plabel_res','order_x').reset_index()
uid_need_pf = order_pf_need.loc[order_pf_need.order_y.notna(),:].groupby(['uid','plabel_res']).order_x.count().reset_index().pivot('uid','plabel_res','order_x').reset_index()
uid_need_pro  = order_pf_need.groupby(['uid','plabel_res','class_bb']).order_x.count().reset_index().pivot_table(index='uid',columns =['plabel_res','class_bb'],values='order_x',aggfunc = np.sum).reset_index()

In [208]:
mid_01  = pd.merge(uid_order,uid_need_pf,how='left',on='uid',suffixes=['_order','_pf'])
mid_02 = pd.merge(mid_01,uid_need_order,how='left',on='uid')
mid_03 = pd.merge(mid_02,uid_need_pro,how='left',on='uid')
mid_04 = pd.merge(mid_03,user_info,how='left',on='uid')
res = pd.merge(mid_04,user_r13,how='left',on='ucode')

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:543: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3108: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [200]:
mid_05.to_excel('12.xlsx')

In [209]:
res.fillna(0,inplace=True)

In [210]:
res.head()

,uid,0_意外险101_order,1_其他产品_order,0_意外险101_pf,1_其他产品_pf,0_意外险101,1_其他产品,"(0_意外险101, 1)","(0_意外险101, 2)","(0_意外险101, 3)",...,"(0_意外险101, 6)","(1_其他产品, 1)","(1_其他产品, 2)","(1_其他产品, 3)","(1_其他产品, 4)","(1_其他产品, 5)","(1_其他产品, 6)",ucode,long_attest,r13
0,00030dc819cf407ca1207d7c0ee74c0a,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1164.0,0
1,00036573a8fd413488ae0463351defc9,7.0,8.0,0.0,0.0,1.0,0.0,3.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1315.0,0
2,0003764068294693af8f72c91ea2fcfd,6.0,39.0,0.0,0.0,0.0,31.0,3.0,0.0,3.0,...,0.0,9.0,3.0,2.0,0.0,0.0,0.0,640269006,380.0,0
3,000459ac1bd14054a1cd66c3e3151ca0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0,389.0,0
4,000467be8f4c4baea8ccf57c7afc2854,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,641.0,0


In [220]:
res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79935 entries, 0 to 79934
Data columns (total 23 columns):
uid               79935 non-null object
0_意外险101_order    79935 non-null float64
1_其他产品_order      79935 non-null float64
0_意外险101_pf       79935 non-null float64
1_其他产品_pf         79935 non-null float64
0_意外险101          79935 non-null float64
1_其他产品            79935 non-null float64
(0_意外险101, 1)     79935 non-null float64
(0_意外险101, 2)     79935 non-null float64
(0_意外险101, 3)     79935 non-null float64
(0_意外险101, 4)     79935 non-null float64
(0_意外险101, 5)     79935 non-null float64
(0_意外险101, 6)     79935 non-null float64
(1_其他产品, 1)       79935 non-null float64
(1_其他产品, 2)       79935 non-null float64
(1_其他产品, 3)       79935 non-null float64
(1_其他产品, 4)       79935 non-null float64
(1_其他产品, 5)       79935 non-null float64
(1_其他产品, 6)       79935 non-null float64
ucode             79935 non-null object
long_attest       79935 non-null float64
r13               79935 non-null

In [216]:
res['r13_bak'] = res.r13.replace('%','',regex=True)
res['r13_bak'] = res.r13_bak.map(lambda x:float(x)/100)

In [221]:
res['is_insider'] = np.nan
res.loc[res.ucode != '0','is_insider'] ='否'
res.is_insider.fillna('是',inplace=True)

In [228]:
res.columns

Index([            'uid',  '0_意外险101_order',    '1_其他产品_order',
           '0_意外险101_pf',       '1_其他产品_pf',        '0_意外险101',
                '1_其他产品', ('0_意外险101', '1'), ('0_意外险101', '2'),
       ('0_意外险101', '3'), ('0_意外险101', '4'), ('0_意外险101', '5'),
       ('0_意外险101', '6'),   ('1_其他产品', '1'),   ('1_其他产品', '2'),
         ('1_其他产品', '3'),   ('1_其他产品', '4'),   ('1_其他产品', '5'),
         ('1_其他产品', '6'),           'ucode',     'long_attest',
                   'r13',         'r13_bak',      'is_insider',
           'rate_pf_acc',   'rate_pf_noacc',   'rete_cust_acc',
       'rete_cust_noacc'],
      dtype='object')

In [226]:
res['rate_pf_acc'] = res['0_意外险101_pf']/res['0_意外险101_order']
res['rate_pf_noacc'] = res['1_其他产品_pf']/res['1_其他产品_order']
res['rete_cust_acc'] = res[('0_意外险101', '1')] + res[('0_意外险101', '2')]+ res[('0_意外险101', '3')]/ res[('0_意外险101', '1')] + \
+ res[('0_意外险101', '2')]+ res[('0_意外险101', '3')]+ res[('0_意外险101', '4')]+ res[('0_意外险101', '5')]+ res[('0_意外险101', '6')]
res['rete_cust_noacc'] = res[('1_其他产品', '1')] + res[('1_其他产品', '2')]+ res[('1_其他产品', '3')]/ res[('1_其他产品', '1')] + \
+ res[('1_其他产品', '2')]+ res[('1_其他产品', '3')]+ res[('1_其他产品', '4')]+ res[('1_其他产品', '5')]+ res[('1_其他产品', '6')]

In [247]:
## 意外险件数
cnt = 5
## 1-3类占比 
rate1_3 = 0.7
## 件数理赔率
rate_pf = 0.01
## 认证时长
long_att = 366

res_01 = res.loc[(res['0_意外险101'] >= cnt) & (res.rete_cust_acc >=rate1_3) & (res.rate_pf_acc <= rate_pf)
                   & (res.is_insider =='否'),['uid']].copy()
res_02 = res.loc[(res['0_意外险101'] >= cnt) & (res.rete_cust_acc >=rate1_3)  & (res.rate_pf_acc <= rate_pf)
                  & (res.is_insider =='是') & (res.long_attest <= long_att),['uid']].copy()
res_03 = res.loc[(res['0_意外险101'] >= cnt) & (res.rete_cust_acc >=rate1_3) & (res.rate_pf_acc <= rate_pf) 
                  & (res.is_insider =='是') & (res.long_attest >= long_att & (res.r13_bak >= 0.9)),['uid']].copy()
res_all_acc = pd.concat([res_01,res_02,res_03]).drop_duplicates()
res_all_acc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8553 entries, 13 to 79925
Data columns (total 1 columns):
uid    8553 non-null object
dtypes: object(1)
memory usage: 133.6+ KB


In [248]:
## 意外险件数
cnt = 0
## 非意外险件数
cnt_noacc = 5
## 1-3类占比 
rate1_3 = 0.7
## 件数理赔率
rate_pf = 0.01
## 认证时长
long_att = 366

res_01 = res.loc[(res['0_意外险101'] <= cnt) & (res['1_其他产品'] >= cnt_noacc ) & (res.rete_cust_noacc >=rate1_3) & (res.rate_pf_noacc <= rate_pf)
                   & (res.is_insider =='否'),['uid']].copy()
res_02 = res.loc[(res['0_意外险101'] <= cnt) & (res['1_其他产品'] >= cnt_noacc ) & (res.rete_cust_noacc >=rate1_3)  & (res.rate_pf_noacc <= rate_pf)
                  & (res.is_insider =='是') & (res.long_attest <= long_att),['uid']].copy()
res_03 = res.loc[(res['0_意外险101'] <= cnt) & (res['1_其他产品'] >= cnt_noacc ) & (res.rete_cust_noacc >=rate1_3) & (res.rate_pf_noacc <= rate_pf) 
                  & (res.is_insider =='是') & (res.long_attest >= long_att & (res.r13_bak >= 0.9)),['uid']].copy()
res_all_noacc = pd.concat([res_01,res_02,res_03]).drop_duplicates()
res_all_noacc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5561 entries, 2 to 79883
Data columns (total 1 columns):
uid    5561 non-null object
dtypes: object(1)
memory usage: 86.9+ KB


In [250]:
res = pd.concat([res_all_acc,res_all_noacc]).drop_duplicates()

In [252]:
order_all.columns

Index(['order', 'order_origin', 'date_pay', 'bf', 'uid', 'pname', 'time_start',
       'time_end', 'time_protect'],
      dtype='object')

In [262]:
order_all.loc[(order_all.uid.isin(res.uid.unique())) 
              &(order_all.pname.isin(['快保-安联意外保险计划','安联意外保(2020版)'])
            &(order_all.date_pay >= pd.to_datetime('2019-03-26'))),['uid','order_origin']].to_excel('12.xlsx')

In [263]:
res.to_excel('12.xlsx')

## 司外用户入司-产品偏好标签

In [24]:
col = ['order','pname','date_pay','bf','uid']
order_all = pd.concat([order_2019_need[col],order_sum[col]])
## 最近一次交易时间
order_all['rns'] = order_all['date_pay'].groupby(order_all['uid']).rank(ascending=False,method='first')
user_order_first = order_all.loc[order_all.rns ==1,['uid','date_pay']]

In [25]:
agg_col = {'order':'count','bf':'sum'}
sdate = pd.to_datetime('2020-03-24')
edate = pd.to_datetime('2020-04-25')
user_order_30 = order_all.loc[(order_all.date_pay >= sdate) & (order_all.date_pay <= edate)].groupby('uid').agg(agg_col).reset_index()

sdate = pd.to_datetime('2020-01-24')
edate = pd.to_datetime('2020-04-25')
user_order_90 = order_all.loc[(order_all.date_pay >= sdate) & (order_all.date_pay <= edate)].groupby('uid').agg(agg_col).reset_index()

sdate = pd.to_datetime('2019-11-24')
edate = pd.to_datetime('2020-04-25')
user_order_180 = order_all.loc[(order_all.date_pay >= sdate) & (order_all.date_pay <= edate)].groupby('uid').agg(agg_col).reset_index()

In [26]:
sdate = pd.to_datetime('2019-09-01')
edate = pd.to_datetime('2020-03-31')
user_order_pname = order_all.loc[(order_all.date_pay >= sdate) & (order_all.date_pay <= edate)].groupby(['uid','pname']).agg(agg_col).reset_index()

user_order_pname['order_rns'] = user_order_pname['order'].groupby(user_order_pname['uid']).rank(ascending=False,method='first')
user_order_pname['bf_rns'] = user_order_pname['bf'].groupby(user_order_pname['uid']).rank(ascending=False,method='first')

user_order_top3 = user_order_pname.loc[user_order_pname.order_rns <=3,:].pivot('uid','order_rns','pname').reset_index()
user_bf_top3 = user_order_pname.loc[user_order_pname.bf_rns <=3,:].pivot('uid','bf_rns','pname').reset_index()

user_order_301 = pd.merge(user_order_first,user_order_30,how='left',on='uid')
user_order_901 = pd.merge(user_order_301,user_order_90,how='left',on='uid')
user_order_1801 = pd.merge(user_order_901,user_order_180,how='left',on='uid')

user_order_3  = pd.merge(user_order_1801,user_order_top3,how='left',on = 'uid')
user_res  = pd.merge(user_order_3,user_bf_top3,how='left',on = 'uid')

## 近期报表

### A类用户入司报表

In [134]:
user_01 = pd.read_excel(dir_down + '用户管理 (23).xlsx')
user_02 = pd.read_excel(dir_down + '用户管理 (24).xlsx')
cn_col = ['用户ID','手机号','姓名','代理人编码','区域中心']
user= pd.concat([user_01[cn_col],user_02[cn_col]])
user.columns = ['uid','ucell','uname','ucode_need','area_type']

In [31]:
user_todo_join= pd.read_excel(dir_des + '司外用户入司\\待入司司外用户名单_2020.04.xlsx')
user_report   = pd.read_excel(dir_des + '司外用户入司\\待入司司外用户名单_2020.04.xlsx',sheet_name='Sheet2')

In [32]:
user_todo_join['uid'] = user_todo_join.uid.str.strip()
user['uid'] = user.uid.str.strip()

In [33]:
user_need = pd.merge(user_todo_join[['uid','推荐人类别','推荐人机构']],user,how='left',on='uid')

user_need['推荐人机构'] = user_need.推荐人机构.str.replace('大童|法人公司|高客','',regex=True)
user_need['推荐人机构'] = user_need.推荐人机构.str.replace('直属经纪','经纪北京',regex=True)

In [34]:
res = user_need.loc[(user_need.推荐人类别=='司内推荐人') & (user_need.ucode.notna()),:].groupby('推荐人机构').uid.nunique().reset_index()
res.columns = ['区域中心','已增员人力']

user_res = pd.merge(user_report,res,how='left',on=['区域中心'])
user_res.loc[user_res.区域中心 =='总计','已增员人力'] = user_res.已增员人力.sum()

In [35]:
user_res.fillna(0,inplace=True)
user_res['入职率'] = (user_res.已增员人力 / user_res.待增员人力).map(lambda x:"{:.2%}".format(x))
user_res[['区域中心', '待增员人力', '已增员人力', '入职率']].to_excel('A类用户已入司情况.xlsx',index=0)

### BC类用户入司报表

In [86]:
user_get =pd.read_excel(dir_des + '司外用户入司\\互联网-t_activity_sign_up_info_0428-04.xlsx')

In [87]:
user_get['user_type'] = np.nan
user_get.loc[user_get.工号.notna(),'user_type'] = '司内用户'
user_get.user_type.fillna('司外用户',inplace=True)

In [88]:
user_get.loc[user_get.推荐人ID =='U00000000','推荐人ID'] = np.nan
user_get['tj_type'] = np.nan
user_get.loc[user_get.推荐人ID.notna(),'tj_type'] = '司外推荐人'
user_get.loc[user_get.推荐人工号.notna(),'tj_type'] = '司内推荐人'
user_get.tj_type.fillna('无推荐人',inplace=True)

In [89]:
user_get.loc[user_get.有效认证==1,'user_cer'] = '认证用户'
user_get.user_cer.fillna('注册用户',inplace=True)

In [90]:
user_get['date_invite'] = pd.to_datetime(user_get.邀请时间).dt.strftime('%Y-%m-%d')
user_get.loc[user_get.date_invite =='2020-04-19','date_invite'] = '2020-04-20'

In [91]:
user_get['user_type_res'] = np.nan
user_get.loc[user_get.user_type =='司内用户','user_type_res'] = '0_司内用户'
user_get.loc[user_get.user_cer =='注册用户','user_type_res'] = '注册用户'
user_get.loc[user_get.user_type_res.isna(),'user_type_res'] = user_get.loc[user_get.user_type_res.isna(),'tj_type']

In [92]:
user_get.groupby(['date_invite','user_type_res']).size().reset_index().pivot(index='user_type_res',columns = 'date_invite',values=0)

date_invite,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28
user_type_res,,,,,,,,,
0_司内用户,3640,1429,779,370,199,35,79,170,438
司内推荐人,144,67,87,45,38,11,14,25,54
司外推荐人,205,119,258,234,110,40,66,58,70
无推荐人,62,30,120,75,36,8,15,11,22
注册用户,160,75,52,40,79,17,17,42,155


### 广告页面数据

In [100]:
data = pd.read_csv(dir_down + 'query_result (9).csv')
data['uid'] = data.uid_cnt

In [101]:
res1= data.groupby(['date','type']).agg({'uid':'nunique','uid_cnt':'count'}).T.stack().reset_index()
res2 = data.groupby(['type']).agg({'uid':'nunique','uid_cnt':'count'}).T.stack().reset_index()
pd.merge(res2,res1,how='left',on=['level_0','type']).sort_values('type')

,level_0,type,0,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24
0,uid,0_页面访问量,17646,7700,5817,5634,4437,1142
5,uid_cnt,0_页面访问量,105557,45952,23075,20477,12802,3251
1,uid,1_启屏,7755,2120,2435,2580,2361,704
6,uid_cnt,1_启屏,12157,2546,2945,3116,2782,768
2,uid,2_弹窗,8856,3935,2186,2692,1417,2329
7,uid_cnt,2_弹窗,15772,5400,2519,3629,1630,2594
3,uid,3_轮播,1006,451,178,262,153,33
8,uid_cnt,3_轮播,1305,587,201,304,177,36
4,uid,4_活动栏,4935,2035,1501,1222,624,180
9,uid_cnt,4_活动栏,8654,3284,2220,1970,911,269


### 智能外呼数据

In [280]:
call = pd.read_excel('D:\\User\\Documents\\WeChat Files\\DNLLAU_ANONY\\FileStorage\\File\\2020-04\\自动外呼清单2020-04-22 13_47_18.xls')

In [282]:
call.head()

,外呼Id,客户姓名,客户号码,外呼状态,创建时间,外显号码编号,语音文件,接通短信模板,未接通短信模板,最大拨打次数,拨打时间,结束时间
0,20200421519976,37496,17600965688,0,2020-04-21 14:26:26,8,ewh.wav,ST00136,ST00121,1,2020-04-22 13:45:42,2020-04-22 13:46:06
1,20200421519991,37497,17710533861,0,2020-04-21 14:26:26,8,ewh.wav,ST00136,ST00121,1,2020-04-22 13:45:49,2020-04-22 13:46:19
2,20200421520006,37498,15033667129,1,2020-04-21 14:26:26,8,ewh.wav,ST00136,ST00121,1,2020-04-22 13:45:51,2020-04-22 13:46:11
3,20200421520021,37499,15884746989,1,2020-04-21 14:26:26,8,ewh.wav,ST00136,ST00121,1,2020-04-22 13:45:58,2020-04-22 13:46:16
4,20200421520036,37500,13508189678,1,2020-04-21 14:26:26,8,ewh.wav,ST00136,ST00121,1,2020-04-22 13:46:00,2020-04-22 13:46:18


### 寿险保单数据

In [178]:
user['ucode'] = user.ucode.map(lambda x:str(x).strip())

In [133]:
df = pd.read_excel(dir_des + 'ReportList_20200421_142051.xlsx',header=1)

In [153]:
on_zj = order_20.loc[(order_20.plabel_term ==1) 
                   & (order_20.plabel_need.str.contains('重疾'))].groupby('ucode').agg({'order':'count','bf':'sum'}).reset_index()

In [154]:
off_zj = df.groupby(['代理人代码']).标准保费.sum().reset_index()
off_zj.columns =[ 'ucode','bf_std']

In [155]:
on_zj['ucode'] = on_zj.ucode.map(lambda x:str(int(x)).strip())
off_zj['ucode'] = off_zj.ucode.map(lambda  x:str(int(x)).strip())

In [158]:
ucode_res = pd.merge(on_zj,off_zj,how='left',on='ucode')

In [181]:
res = pd.merge(ucode_res,user,how='left',on='ucode')

In [185]:
user_insider = pd.read_excel(dir_user + 'ReportList_20200421_160639.xlsx')

In [191]:
user_team_leader = user_insider.loc[user_insider.经营合伙人代码.notna(),['经营合伙人代码','经营合伙人姓名']].drop_duplicates()
user_team_leader.columns = ['ucode','uname']
user_team_leader['ucode'] = user_team_leader.ucode.map(lambda x:str(int(x)).strip())

In [195]:
res_need = pd.merge(res,user_team_leader,how='left',on='ucode')

In [197]:
res_need.to_excel('12.xlsx')

In [165]:
order_sum.groupby(['mon','is_insider']).uid.nunique()

mon      is_insider
2020-01  否              8798
         是             13276
2020-02  否              6961
         是             14296
2020-03  否              8585
         是             15064
2020-04  否              8377
         是             12192
Name: uid, dtype: int64

In [167]:
order_life = pd.read_excel(dir_des + 'ReportList_20200424_140050.xlsx',header=1)

In [169]:
order_life.columns

Index(['序号', '保单状态', '退保日期', '回单日期', '保单录入日期', '承保日期', '预收日期', '行政录入日期',
       '销售团队代码', '销售团队名称', '地区分类', '团队分类', '是否本部', '营业部代码', '营业部名称', '是否营业中心',
       '营业部所属批牌机构编码', '营业部所属批牌', '营业部类型', '代理人代码', '代理人姓名', '入司日期', '代理人职级',
       '业务合伙人级别', '经营合伙人级别', '经营团队总经理标志', '推荐人代码', '推荐人姓名', '辅导人代码', '辅导人姓名',
       '经营合伙人代码', '经营合伙人姓名', '离职日期', '现代理人编码', '现代理人姓名', '代理人所在机构', '原辅代理人姓名',
       '原辅代理人编码', '原辅代理人所在销售团队名称', '原辅代理人所在销售团队代码', '原辅代理人所在机构', '现辅代理人姓名',
       '现辅代理人编码', '现辅代理人所在销售团队名称', '现辅代理人所在销售团队代码', '是否自保件', '保险公司代码',
       '保险公司名称', '投保单号', '保单号', '投保人', '被保险人', '险种代码', '外部险种代码', '险种名称',
       '是否主险', '产品分类', '交费年期|年龄标志', '缴费期', '佣金率', '规模保费', '标准保费', '保额', '保全标志',
       '是否纳入现代理人方案统计', '归属类型', '归属细类', '保单关系人', '代码', '姓名', '月份', '销服代码',
       '销服名称', '机构', '职级', '入司日期.1', '入司月', '营业部代码.1', '营业部名称.1', '是否营业中心.1',
       '师傅代码', '师傅姓名'],
      dtype='object')

In [173]:
order_life['mon'] = order_life.行政录入日期.map(date_bus)
order_life_res = order_life[['mon','代理人代码','标准保费']].groupby(['mon','代理人代码']).标准保费.sum().reset_index()
order_life_res.columns = ['mon','ucode','bf_std']

In [174]:
order_online =  order_sum.groupby(['mon','ucode']).bf.sum().reset_index()

In [176]:
order_life_res['ucode'] = order_life_res.ucode.map(lambda x:str(int(x)))
order_online['ucode'] = order_online.ucode.map(lambda x:str(int(x)))

In [182]:
order_life_res.groupby('mon').ucode.nunique()

mon
2020-01    6472
2020-02    3458
2020-03    5558
2020-04    5575
Name: ucode, dtype: int64

In [180]:
res = pd.merge(order_online,order_life_res,how='left',on=['mon','ucode'])
res.loc[res.mon == '2020-03'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15079 entries, 27593 to 42671
Data columns (total 4 columns):
mon       15079 non-null object
ucode     15079 non-null object
bf        15079 non-null float64
bf_std    4500 non-null float64
dtypes: float64(2), object(2)
memory usage: 589.0+ KB


In [190]:
order_2019_need.loc[(order_2019_need.date_pay >= pd.to_datetime('2019-06-26'))
                    & (order_2019_need.area_type == '大童青岛区域中心'),:].groupby(['ucode','uname']).bf.sum().reset_index().sort_values('bf',ascending=False)

,ucode,uname,bf
62,640201727.0,王菁,180861.50
124,640225216.0,徐立群,97968.20
334,640272931.0,应芷欣,73141.20
5,640179111.0,赵丽丽,49813.00
57,640200237.0,闫文奇,48915.18
188,640246099.0,孙海叶,41882.30
145,640229495.0,张玉兴,40091.28
38,640192529.0,陈会升,38669.00
105,640222596.0,王振,38181.00
8,640179753.0,马玉涛,35773.70


## 离职代理人清单

In [236]:
user_in = pd.read_excel(dir_user + 'ReportList_20200426_134954.xlsx',header=2)

In [324]:
user_trans = pd.read_excel(dir_user+ '服务助理转换.xlsx')

In [325]:
user_need_in = user_in[['代理人编码','Unnamed: 35']].copy()
user_need_in.columns = ['ucode','date_left']

In [326]:
order_sum.ucode.fillna(0,inplace=True)
order_sum['ucode'] = order_sum.ucode.map(lambda x:str(int(x)).strip())
user_need_in.ucode.replace('[a-zA-Z]','',inplace=True,regex=True)
user_need_in['ucode'] = user_need_in.ucode.map(lambda x:str(int(x)).strip())
user_trans['ucode'] = user_trans.ucode.map(lambda x:str(int(x)).strip())
user_trans['ucode_new'] = user_trans.ucode_new.map(lambda x:str(int(x)).strip())

In [300]:
user_trans = pd.merge(user_trans,user_need_in,how='left',left_on='ucode_new',right_on='ucode')

In [302]:
user_trans['user_type'] = np.nan
user_trans.loc[user_trans.date_left.isna(),'user_type'] = '在职'
user_trans.user_type.fillna('离职',inplace=True)

In [303]:
user_need_in['user_type'] = np.nan
user_need_in.loc[user_need_in.date_left.isna(),'user_type'] = '在职'
user_need_in.user_type.fillna('离职',inplace=True)

In [305]:
user_need_in.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138783 entries, 1 to 138783
Data columns (total 3 columns):
ucode        138783 non-null object
date_left    95210 non-null datetime64[ns]
user_type    138783 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 4.2+ MB


In [306]:
user_trans.info(0)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7650 entries, 0 to 7649
Data columns (total 5 columns):
ucode_x      7650 non-null object
ucode_new    7650 non-null object
ucode_y      7650 non-null object
date_left    2240 non-null datetime64[ns]
user_type    7650 non-null object
dtypes: datetime64[ns](1), object(4)
memory usage: 358.6+ KB


In [308]:
user_res = pd.merge(user_need_in,user_trans[['ucode_x','user_type']],how='left',left_on='ucode',right_on = 'ucode_x')

In [313]:
user_res.loc[user_res.user_type_y.isna(),'user_type_y'] = user_res.loc[user_res.user_type_y.isna(),'user_type_x'] 

In [314]:
user_res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138784 entries, 0 to 138783
Data columns (total 5 columns):
ucode          138784 non-null object
date_left      95211 non-null datetime64[ns]
user_type_x    138784 non-null object
ucode_x        7650 non-null object
user_type_y    138784 non-null object
dtypes: datetime64[ns](1), object(4)
memory usage: 6.4+ MB


In [315]:
user_res.groupby('user_type_y').size().reset_index()

,user_type_y,0
0,在职,48983
1,离职,89801


In [322]:
order_sum[['ucode']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522334 entries, 0 to 526255
Data columns (total 1 columns):
ucode    396528 non-null float64
dtypes: float64(1)
memory usage: 8.0 MB


In [328]:
res = pd.merge(order_sum,user_res[['ucode','date_left','user_type_y']],how='left',on='ucode')

In [332]:
order_sum.loc[order_sum.is_insider =='是'].groupby('mon').bf.sum().reset_index()

,mon,bf
0,2020-01,2.975547e+07
1,2020-02,3.229185e+07
2,2020-03,2.978988e+07
3,2020-04,3.282895e+07


In [333]:
order_2019_need.ucode.fillna(0,inplace=True)
order_2019_need['ucode'] = order_2019_need.ucode.map(lambda x:str(int(x)).strip())

In [42]:
col = ['mon','date_pay','bf','is_insider','uid','ucode','pname','bb_id']
order_all = pd.concat([order_2019_need[col],order_sum[col]])

In [351]:
res1 = pd.merge(order_all,user_res[['ucode','date_left','user_type_y']],how='left',on='ucode')

In [352]:
order_ptype = pd.merge(res1,ptype,how='left',on='pname')

In [357]:
order_ptype.loc[order_ptype.user_type_y =='离职'].groupby(['mon','plabel_term']).bf.sum().reset_index().pivot(index='mon',columns = 'plabel_term',values='bf').to_excel('12.xlsx')

In [355]:
order_ptype.loc[(order_ptype.is_insider.isin(['是','否'])) 
                & (order_ptype.uid.isin(order_ptype.loc[order_ptype.user_type_y =='离职'].uid.unique()))].groupby(['mon']).bf.sum().reset_index()

,mon,bf
0,2019-01,3890411.14
1,2019-02,2948792.19
2,2019-03,4084045.94
3,2019-04,4106498.82
4,2019-05,3180393.44
5,2019-06,3228044.77
6,2019-07,2487065.25
7,2019-08,2853019.30
8,2019-09,3217095.01
9,2019-10,2362743.25


In [349]:
order_ptype.loc[order_ptype.user_type_y =='离职'].uid.nunique()

AttributeError: 'DataFrame' object has no attribute 'uid'

In [365]:
res = pd.merge(order_sum,ptype,how='left',on='pname')

In [373]:
res.loc[res.plabel_L1.isin(['意外险','健康险'])].groupby(['mon','plabel_L1','is_renew']).bf.sum().reset_index().pivot_table(index='mon',columns = ['plabel_L1','is_renew'],values='bf')

plabel_L1          健康险                      意外险            
is_renew             0            1           0           1
mon                                                        
2020-01    10061030.63   7989873.60  6567731.56  3221131.08
2020-02     9861568.98   9100836.69  8961125.00  2852965.40
2020-03     9159220.49   9889400.90  6595926.16  4831572.78
2020-04    10407483.96  10497677.29  8210306.60  5031252.84

### 待入司司外用户名单

In [18]:
user = pd.read_excel('D:\\User\\Desktop\\司外用户入司\\A类用户\\待入司司外用户名单.xlsx')

In [30]:
user.rename(columns = {'用户ID':'uid'},inplace=True)

In [31]:
user_res.head()

,uid,date_pay,order_x,bf_x,order_y,bf_y,order,bf,1.0_x,2.0_x,3.0_x,1.0_y,2.0_y,3.0_y
0,05aec1da1ec049a6b0da280aceb4567a,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ac8b6943b0954e31bb7eea8bc0d063c6,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3d25b2abdd2b46d1af4598106235bedd,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5a779ca863c148f183f5a6c6d66acc2e,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9672df71663d455eb8705a16e3c5179f,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
res.loc[res.工号.isna(),:].to_excel('12.xlsx')

In [32]:
res = pd.merge(user,user_res,how='left',on='uid')

In [38]:
df = pd.read_csv(dir_down + 'query_result (11).csv',low_memory=False)

In [36]:
df1 = pd.read_clipboard()

In [46]:
res1 = order_all.groupby(['uid']).bb_id.nunique().reset_index()

In [44]:
res = pd.merge(df1,df,how='left',on='uid')[['uid','最近一次登录时间','最近一次交易时间']]

In [48]:
res2 = pd.merge(res,res1,how='left',on='uid')

In [49]:
df2 = pd.read_csv(dir_down + 'query_result (13).csv',low_memory=False)

In [50]:
df2.columns

Index(['uid', '是否司内用户', '评分r1-最近一次登录时间距离当日的天数', '评分f1-周期内登录的天数',
       '评分r2-最近一次交易时间距离当日的天数', '评分f2-周期内交易件数', '评分m-周期内总保费', '用户综合评分',
       '用户综合价值', '用户认证时间', '用户注册时间', '最近一次登录时间', '最近一次交易时间', '最近一次登录时间距离当日的天数',
       '周期内登录的天数', '最近一次交易时间距离当日的天数', '周期内交易件数', '周期内交易天数', '周期内总保费'],
      dtype='object')

In [53]:
pd.merge(res2,df2[['uid','用户综合价值']],how='left',on='uid').to_excel('11.xlsx')

Exception ignored in: <bound method Workbook.__del__ of <xlsxwriter.workbook.Workbook object at 0x0000027312DDD278>>
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\workbook.py", line 154, in __del__
    raise Exception("Exception caught in workbook destructor. "
Exception: Exception caught in workbook destructor. Explicit close() may be required for workbook.


In [57]:
order_2019_need.loc[order_2019_need.uname == '徐立群',['uname','uid','ucode','area_type']].groupby(['uid','ucode']).size().reset_index()

,uid,ucode,0
0,728d5b5d31304b96aa79015cd5a97b61,640225216.0,212


In [54]:
order_sum.loc[order_sum.area_type =='大童青岛区域中心',:].groupby(['ucode','uname']).bf.sum().reset_index().sort_values('bf',ascending=False)

,ucode,uname,bf
53,640201727.0,王菁,117283.00
257,640275229.0,张强,80277.79
86,640222596.0,王振,61842.80
245,640274213.0,王海涛,49128.90
90,640223243.0,张中辉,41766.60
18,640184360.0,陈晓燕,37216.70
262,640275442.0,汪美奇,35387.60
55,640206357.0,郑雨静,34478.00
4,640179111.0,赵丽丽,33670.40
111,640229228.0,单清辰,32685.00


In [58]:
df1 = pd.read_excel(dir_down + '用户订单.xlsx')
df2 = pd.read_excel(dir_down + '定期寿订单管理 (2).xlsx')

In [63]:
col = ['订单号','保单号','订单状态','支付日期','保单金额','推广费代理人','投保人']
res = pd.concat([df1[col],df2[col]])

In [65]:
res['date_pay'] = pd.to_datetime(pd.to_datetime(res.支付日期).dt.strftime('%Y-%m-%d'))

In [80]:
res.loc[(res.date_pay >= pd.to_datetime('2019-06-26')) 
        & (res.date_pay <= pd.to_datetime('2019-12-25'))
       & (res.推广费代理人 =='徐立群')].to_excel('12.xlsx')

In [70]:
res.head()

,订单号,保单号,订单状态,支付日期,保单金额,推广费代理人,投保人,date_pay
0,202004264310928258,361012006000053800,已生效,2020-04-26 10:28:49,80.0,周海梅,张立立,2020-04-26
1,202004259813111245,10103006600041219306,待生效,2020-04-25 12:22:25,115.0,周海梅,张雪冬,2020-04-25
2,202004236756901246,P000000002333668,待生效,2020-04-23 16:11:29,200.0,徐立群,郭贤龙,2020-04-23
3,202004233245483331,P000000002333660,待生效,2020-04-23 16:07:45,200.0,徐立群,郭东艳,2020-04-23
4,202004236634648180,P000000002333650,待生效,2020-04-23 16:04:06,200.0,徐立群,郑丙军,2020-04-23


## 寿险数据

In [4]:
dir_user = 'E:\\SynologyDrive\\dir_user\\'
dir_order_life = 'E:\\SynologyDrive\\dir_order\\order_life\\'

In [53]:
col = ['uid','date_pay','pname','bf','order','uname','ucell','ucode']
order_online = pd.concat([order_2019_need[col],order_2020_all[col]])

In [54]:
order_all = pd.merge(order_online,ptype,how='left',on='pname')

### 用户数据

### 保单数据

In [72]:
user_order_off = order_detail.groupby(['ucode']).bf_std.sum().reset_index()

In [81]:
user_order_on.fillna(0,inplace=True)
user_order_on['ucode'] = user_order_on.ucode.map(lambda x:str(int(x)).strip())
user_order_off['ucode'] = user_order_off.ucode.map(lambda x:str(int(x)).strip())


In [91]:
res_mid = pd.merge(user_order_on,user_order_off,how='left',on='ucode')
res = pd.merge(res_mid,user_leader,how='left',on='ucode')

In [92]:
res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17520 entries, 0 to 17519
Data columns (total 8 columns):
uid             17520 non-null object
order           17520 non-null int64
bf              17520 non-null float64
ucell           17520 non-null int64
uname           17520 non-null object
ucode           17520 non-null object
bf_std          6688 non-null float64
uname_leader    321 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 1.2+ MB


In [95]:
res.loc[res.uname_leader.isna(),:].to_excel(dir_data_support + '名单 to思思.xlsx')

## 直播数据

In [69]:
user_zb = pd.read_excel('E:\\SynologyDrive\\data_temp\\4.28直播观看名单.xlsx')
user_all = pd.read_csv('E:\\SynologyDrive\\data_temp\\user_info_20200429.csv')

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
user_zb.rename(columns ={'用户ID':'uid','观看时长':'time_long'},inplace=True)

In [43]:
user_all.rename(columns = {'用户ID':'uid','工号':'ucode','推荐人ID':'tj_uid','大童工号':'tj_ucode','注册时间':'time_regist'},inplace=True)

In [23]:
user_get =pd.read_excel(dir_des + '司外用户入司\\互联网-t_activity_sign_up_info_0428-04.xlsx')

In [27]:
user_get_01 = user_get[['用户ID','邀请码']].copy()
user_get_01.columns = ['uid','code_invite']

In [71]:
user_zb_01 = user_zb[['uid','time_long']].copy()

In [44]:
user_all_01 = user_all[['uid','time_regist','ucode','tj_uid','tj_ucode']].copy()

In [72]:
user_res = pd.merge(user_zb_01,user_all_01,how='left',on='uid')

In [73]:
user_res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2907 entries, 0 to 2906
Data columns (total 6 columns):
uid            2907 non-null object
time_long      2907 non-null int64
time_regist    2768 non-null object
ucode          2478 non-null float64
tj_uid         2656 non-null object
tj_ucode       1824 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 159.0+ KB


In [74]:
user_res.loc[user_res.ucode.isna(),:].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 429 entries, 6 to 2905
Data columns (total 6 columns):
uid            429 non-null object
time_long      429 non-null int64
time_regist    290 non-null object
ucode          0 non-null float64
tj_uid         223 non-null object
tj_ucode       89 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 23.5+ KB


In [75]:
user_res.loc[(user_res.time_regist.notna()) & (user_res.ucode.isna()),:].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290 entries, 6 to 2905
Data columns (total 6 columns):
uid            290 non-null object
time_long      290 non-null int64
time_regist    290 non-null object
ucode          0 non-null float64
tj_uid         223 non-null object
tj_ucode       89 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 15.9+ KB


In [79]:
res = pd.merge(user_get_01,user_all_01,how='left',on='uid')

In [80]:
res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9800 entries, 0 to 9799
Data columns (total 6 columns):
uid            9785 non-null object
code_invite    9800 non-null int64
time_regist    9169 non-null object
ucode          7147 non-null float64
tj_uid         8681 non-null object
tj_ucode       5626 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 535.9+ KB


In [81]:
res.loc[res.tj_uid =='U00000000','tj_uid'] = np.nan

In [82]:
res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9800 entries, 0 to 9799
Data columns (total 6 columns):
uid            9785 non-null object
code_invite    9800 non-null int64
time_regist    9169 non-null object
ucode          7147 non-null float64
tj_uid         8596 non-null object
tj_ucode       5626 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 535.9+ KB


In [85]:
res.loc[(res.ucode.isna()) & (res.tj_ucode.isna()) & (res.time_regist.notna()),:].to_excel('12.xlsx')

In [3]:
user = pd.read_clipboard()

D:\ProgramData\Anaconda3\lib\site-packages\pandas\io\clipboards.py:63: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_table(StringIO(text), sep=sep, **kwargs)


In [5]:
user_info = pd.read_csv(dir_down + 'query_result (14).csv')

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
user_info.head()

,uid,是否司内用户,评分r1-最近一次登录时间距离当日的天数,评分f1-周期内登录的天数,评分r2-最近一次交易时间距离当日的天数,评分f2-周期内交易件数,评分m-周期内总保费,用户综合评分,用户综合价值,用户认证时间,用户注册时间,最近一次登录时间,最近一次交易时间,最近一次登录时间距离当日的天数,周期内登录的天数,最近一次交易时间距离当日的天数,周期内交易件数,周期内交易天数,周期内总保费
0,4e785d1ca4864fc1a7f0621a7b52581e,0,4,1,0,0,0,30.0,1,2019-04-22 17:06:22,2019-04-03 20:34:16,2020-04-30,NaN,6.0,1.0,NaN,NaN,NaN,NaN
1,e0381c916ed8450ebf6aec5dc030c10b,1,4,2,2,1,2,185.0,2,2017-11-24 21:15:33,2017-11-18 23:56:37,2020-04-29,2020-04-01,7.0,12.0,35.0,1.0,1.0,550.0
2,fc75e0531b87414d8895acc331a60020,0,3,1,0,0,0,25.0,1,2018-06-07 16:29:00,2018-06-07 16:29:00,2020-04-06,NaN,30.0,5.0,NaN,NaN,NaN,NaN
3,f3356828078643b6b3096b1e5778d239,1,5,4,0,0,0,65.0,1,2018-01-18 10:42:28,2018-01-18 10:29:41,2020-05-05,NaN,1.0,33.0,NaN,NaN,NaN,NaN
4,1a2712435dd048eca33177a42c936211,1,5,4,5,4,3,385.0,4,2017-09-14 11:27:52,2017-09-14 10:50:01,2020-05-05,2020-05-05,1.0,35.0,1.0,6.0,3.0,1570.0


In [8]:
res_mid = pd.merge(user,user_info[['uid','是否司内用户','最近一次登录时间','最近一次登录时间距离当日的天数']],how='left',on='uid')

In [11]:
res = pd.merge(res_mid,user_info[['uid','是否司内用户']],how='left',left_on='tj_uid',right_on ='uid')

In [13]:
res.head()

,uid_x,tj_uid,是否司内用户_x,最近一次登录时间,最近一次登录时间距离当日的天数,uid_y,是否司内用户_y
0,f969c01dd5944d3aa5aec98be6cb0ac2,f6b57dce01864769968398a026c4ad83,0.0,NaN,NaN,f6b57dce01864769968398a026c4ad83,1.0
1,f5aee48c1f124127a76f464f768e6afd,f6b57dce01864769968398a026c4ad83,0.0,NaN,NaN,f6b57dce01864769968398a026c4ad83,1.0
2,e33d7d57e00e46e5a64fadae24d73361,f6b57dce01864769968398a026c4ad83,0.0,2020-05-01,5.0,f6b57dce01864769968398a026c4ad83,1.0
3,df264d1d92264f3d9a6e121d3dac780a,f6b57dce01864769968398a026c4ad83,0.0,NaN,NaN,f6b57dce01864769968398a026c4ad83,1.0
4,dfeceab773904751ab8f9e50d397951a,f6b57dce01864769968398a026c4ad83,0.0,NaN,NaN,f6b57dce01864769968398a026c4ad83,1.0


In [23]:
res.loc[(res.是否司内用户_x==0) 
        & (res.是否司内用户_y !=1)
        &(res.最近一次登录时间.notna()),:].to_excel('12.xlsx')

## 核心指标

In [47]:
order_2020 = pd.read_csv('D:\\User\\Desktop\\order_info\\his\\order_report_0420.csv',encoding='utf-8',parse_dates=['date_pay'],low_memory=False)
order_20 = order_2020.loc[(order_2020.order_stat.isin(['待生效','已生效','已发货','契撤']))
                         & (order_2020.date_pay <= pd.to_datetime('2019-12-25'))].copy()

In [48]:
order_20.order_stat.unique()

array(['待生效', '已生效', '已发货', '契撤'], dtype=object)

In [49]:
order_2020.order_stat.unique()

array(['待生效', '已生效', '撤保', '退保', '已发货', '订单关闭', '已取消', '契撤'], dtype=object)

In [50]:
order_20['mon'] = order_20.date_pay.map(date_bus)

In [52]:
order_20.groupby('mon').bf.sum().reset_index()

,mon,bf
0,2019-01,3.050527e+07
1,2019-02,2.490442e+07
2,2019-03,3.616098e+07
3,2019-04,3.896023e+07
4,2019-05,3.346649e+07
5,2019-06,3.556558e+07
6,2019-07,3.158858e+07
7,2019-08,3.489812e+07
8,2019-09,4.002310e+07
9,2019-10,3.322544e+07


In [41]:
order_21 = order_2020.loc[(order_2020.order_stat.isin(['待生效','已生效','已发货']))
                         & (order_2020.date_pay >= pd.to_datetime('2019-12-26'))].copy()

In [42]:
order_21['mon'] = order_21.date_pay.map(date_bus)

In [43]:
order_21.groupby('mon').bf.sum().reset_index()

,mon,bf
0,2020-01,37939980.10
1,2020-02,38722899.55
2,2020-03,38098268.85
3,2020-04,34497254.13


In [56]:
order_20.groupby('is_insider').bf.sum().reset_index()

,is_insider,bf
0,否,1.047130e+08
1,是,3.100099e+08


In [55]:
order_20.loc[order_20.bf <=0,:].groupby('pname').size().reset_index()

,pname,0
0,安联国庆节公共交通意外保障计划,395
1,安联少儿意外险,1
2,永安“离家无忧”短期家财险,1


In [36]:
order_20.groupby('date_pay').bf.sum().reset_index()

,date_pay,bf
0,2018-12-26,1137401.52
1,2018-12-27,1178226.55
2,2018-12-28,1037094.96
3,2018-12-29,1001030.21
4,2018-12-30,794357.91
5,2018-12-31,935097.50
6,2019-01-01,715735.30
7,2019-01-02,1225583.68
8,2019-01-03,1271064.00
9,2019-01-04,1278249.74


In [40]:
za_order = pd.read_clipboard()

D:\ProgramData\Anaconda3\lib\site-packages\pandas\io\clipboards.py:63: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_table(StringIO(text), sep=sep, **kwargs)


In [44]:
res = pd.merge(za_order,order_20[['order_origin','bf']],how='left',on='order_origin')

In [57]:
res.loc[res.order_origin =='IH1100351698437124',:]

,order_origin,bf_za,bf
7801,IH1100351698437124,2.104109,256.0


In [58]:
res.loc[res.bf.notna(),:].head()

,order_origin,bf_za,bf
11,IH1100348125120767,1576.0,1576.0
12,IH1100348128853406,466.0,466.0
13,IH1100348128945530,146.0,146.0
14,IH1100348128496621,176.0,176.0
15,IH1100348133168720,166.0,176.0


In [59]:
need =res.loc[res.bf.isna(),:].order_origin.unique()

In [60]:
need

array(['IH1100348088875928', 'IH1100348089572816', 'IH1100348090709119',
       ..., 'IH1100351947926540', 'IH1100351962163955',
       'IH1100351962662104'], dtype=object)

In [69]:
for x in need:
    need_res = order_20.loc[(order_20.order_origin.str.contains(x))
             & (order_20.order_origin.notna()),:].order.nunique()
    if need_res ==0:
        print(x,need_res)

IH1100349068552713 0
IH1100349067886828 0
IH1100349070731859 0
IH1100349071143607 0
IH1100349070734608 0
IH1100349071147929 0
IH1100349070395038 0
IH1100349070734572 0
IH1100349071899279 0
IH1100349073517516 0
IH1100349073526912 0
IH1100349073186506 0
IH1100349073432687 0
IH1100349074180166 0
IH1100349075074052 0
IH1100349074733751 0
IH1100349074740968 0
IH1100349076009523 0
IH1100349075353448 0
IH1100349078202005 0
IH1100349078215554 0
IH1100349078609134 0
IH1100349078469192 0
IH1100349078469059 0
IH1100349080146473 0
IH1100349079758618 0
IH1100349079412161 0
IH1100349080146340 0
IH1100349080146811 0
IH1100349080594279 0
IH1100349082514352 0
IH1100349083134049 0
IH1100349083298762 0
IH1100349083606994 0
IH1100349083848124 0
IH1100349084199128 0
IH1100349084578152 0
IH1100349084199430 0
IH1100349084578454 0
IH1100349083848390 0
IH1100349084996567 0
IH1100349085271319 0
IH1100349087228688 0
IH1100349089505429 0
IH1100349089911131 0
IH1100349089915888 0
IH1100349090636349 0
IH11003490902

In [54]:
order_20.loc[(order_20.order_origin.str.contains('IH1100351962662104'))
             & (order_20.order_origin.notna()),['order','order_origin','pname','bf']]

,order,order_origin,pname,bf
522556,202004261317909207,"IH1100351962163955,IH1100351962662104",众安尊享e生2020版,731.0


In [66]:
need_res = order_20.loc[(order_20.order_origin.str.contains('IHKB1100351962662104'))
             & (order_20.order_origin.notna()),:].order.nunique()

In [67]:
need_res

0

In [48]:
sdate = pd.to_datetime('2020-03-23')
edate = pd.to_datetime('2020-04-26')
plist = ['众安尊享e生2020版','众安尊享e生2019版','众安尊享e生(优甲版)','众安尊享e生(城市版)','众安尊享e生爸妈版','众安尊享e生旗舰版']
order_20.loc[(order_20.date_pay >= sdate) & (order_20.date_pay <=edate) & (order_20.pname.isin(plist)),:].groupby('pname').bf.sum().reset_index().sort_values('bf',ascending=False)

,pname,bf
3,众安尊享e生2020版,2412830.0
2,众安尊享e生2019版,2177686.0
5,众安尊享e生爸妈版,262072.0
4,众安尊享e生旗舰版,210954.0
0,众安尊享e生(优甲版),23580.0
1,众安尊享e生(城市版),6408.0


In [71]:
order_2020_all.loc[(order_2020_all.order_origin.str.contains('IH1100351816490675'))
             & (order_2020_all.order_origin.notna()),['order','order_origin','pname','bf']]

,order,order_origin,pname,bf


In [40]:
order_20.loc[order_20.plabel_need.isna(),:].groupby('pname').bf.sum().reset_index()

,pname,bf
0,富德生命大富翁年金保险,40000.0


In [44]:
order_20.loc[(order_20.plabel_need.str.contains('重疾')) & (order_20.plabel_term ==1) & (order_20.plabel_need.notna()),:].groupby('uid').agg({'order':'count','bf':'sum'}).to_excel('12.xlsx')

In [47]:
order_20.columns

Index(['order', 'order_origin', 'order_stat', 'time_order', 'time_pay',
       'time_cb', 'time_start', 'time_end', 'pcode', 'pid', 'pname',
       'pcompany', 'plan', 'pay_way', 'channel_src', 'channel_detail',
       'batch_id', 'order_renew_src', 'is_self', 'bf', 'tgf_rate_A',
       'tgf_rate_B', 'tgf_A', 'tgf_B', 'tgf_tax', 'tjf_tax', 'tjf', 'tgf_b1',
       'tgf_b2', 'money_add', 'ticket_code', 'ticket_name', 'ticket_rate',
       'money_top', 'ticket_value', 'tgf_X', 'uid', 'order_origin_uid',
       'uname', 'ucell', 'uprov', 'ucity', 'uarea', 'is_insider', 'ucode',
       'time_join', 'area_type', 'area_work', 'area_service_id',
       'area_service', 'team_sale_id', 'team_sale_name', 'team_bus_id',
       'team_bus_name', 'tj_name', 'tj_uid', 'tj_cell', 'tj_prov', 'tj_city',
       'tj_area', 'tj_area_work', 'tj_area_service', 'tb_name', 'tb_id_type',
       'tb_id', 'tb_email', 'tb_cell', 'relation', 'bb_name', 'bb_id_type',
       'bb_id', 'date_pay', 'mon', 'plabel_L1', 'p

In [49]:
order_20.groupby(['date_pay','plabel_need','pname']).bf.sum().reset_index().to_excel('12.xlsx')